A cute little demo showing the simplest usage of minGPT. Configured to run fine on Macbook Air in like a minute.

In [ ]:
classdef OvernightLegARRWithManualSchedule < H_RootObject
    % FixedRateLeg
    % properties: fixedSchedule 
    
    properties(SetAccess = public)
        nominals
        gearing
        spread
        overnightIndex
        manualSchedule
        liborFloatingSchedule
        calendar
        paymentConvention
%         payLag
%         legSize
        couponRates
    end
    
    methods
        % constructor
        function floatingRateLeg = OvernightLegARRWithManualSchedule(manualSchedule,liborFloatingSchedule,overnightIndex,gearing,spread,nominal,...
                                      dayCounterLibor,dayCounter,paymentConvention,paymentCalendar)
                
                
                floatingRateLeg.manualSchedule = manualSchedule;
                floatingRateLeg.liborFloatingSchedule = liborFloatingSchedule;
                
                floatingRateLeg.calendar = paymentCalendar;
                floatingRateLeg.paymentConvention = paymentConvention;
                
                
                floatingRateLeg.overnightIndex = overnightIndex;
                
                
                legSize = size(manualSchedule,1);
%                 legSize = length(schedule.dates)-1;
%                 floatingRateLeg.legSize = legSize;
                
                floatingRateLeg.spread = zeros(legSize,1);
                floatingRateLeg.nominals = zeros(legSize,1);
                
                for i=1:legSize
                    floatingRateLeg.nominals(i) = nominal;
                    floatingRateLeg.gearing(i) = gearing;
                    floatingRateLeg.spread(i) = spread;
                    
%                     startDate = schedule.dates{i};
%                     endDate = schedule.dates{i+1};
%                     paymentDate = floatingRateLeg.calendar.Advance(endDate,paymentConvention,...
%                                                                     payLag,'day',true);
                    
                    startDate   = manualSchedule{i,1};
                    endDate     = manualSchedule{i,2};
                    paymentDate = manualSchedule{i,3};
                    
                    refPeriodStartDate = liborFloatingSchedule{i,1};
                    refPeriodEndDate   = liborFloatingSchedule{i,2};
                    
                    
                    floatingRateLeg.couponRates{i} = OvernightIndexedCouponARR(paymentDate,nominal,overnightIndex,gearing,spread,dayCounterLibor,dayCounter,...
                                                     startDate,endDate,refPeriodStartDate,refPeriodEndDate);   
                end
                
        end
        
        function out = NPV(floatingRateLeg,discountCurve)
           out = 0;
            for i=1:length(floatingRateLeg.couponRates)
                out = out + floatingRateLeg.couponRates{i}.NPV(discountCurve);
            end
        end
        
        function out = BPS(floatingRateLeg,discountCurve)
           out = 0;
            for i=1:length(floatingRateLeg.couponRates)
                out = out + floatingRateLeg.couponRates{i}.BPS(discountCurve);
            end
        end
        
        function out = NPV_Term(floatingRateLeg,discountCurve,termDate,beforeAfterFlag)
           out = 0;
           idxTerm = 0;
           couponRates = floatingRateLeg.couponRates;
           % Find idxTerm
           % if beforeAfterFlag is before then
           % couponRates whose fixngDate is before(or equal to) termDate
           % are only compouted
           % -> to use the same function for overnightLeg
           % we change fixingDate to accrualStartDate
           % otherwise
           % couponRates whose fixingDate is
           for i=1:length(couponRates)
                if DateDiff(termDate,couponRates{i}.accrualStartDate) < 0
                    break;
                end
                idxTerm = idxTerm + 1;
           end
           
           if strcmp(beforeAfterFlag,'before')
               for i=1:idxTerm
                   out = out + couponRates{i}.NPV(discountCurve);
               end
               
           else
               for i= idxTerm+1:length(couponRates)
                   out = out + couponRates{i}.NPV(discountCurve);
               end
           end
           
%             for i=1:length(floatingRateLeg.couponRates)
%                 out = out + floatingRateLeg.couponRates{i}.NPV(discountCurve);
%             end

        end
        
        function out = BPS_Term(floatingRateLeg,discountCurve,termDate,beforeAfterFlag)
           out = 0;
           idxTerm = 0;
           couponRates = floatingRateLeg.couponRates;
           % Find idxTerm
           % if beforeAfterFlag is before then
           % couponRates whose fixngDate is before(or equal to) termDate
           % are only compouted
           % -> to use the same function for overnightLeg
           % we change fixingDate to accrualStartDate
           % otherwise
           % couponRates whose fixingDate is
           for i=1:length(couponRates)
                if DateDiff(termDate,couponRates{i}.accrualStartDate) < 0
                    break;
                end
                idxTerm = idxTerm + 1;
           end
           
           if strcmp(beforeAfterFlag,'before')
               for i=1:idxTerm
                   out = out + couponRates{i}.BPS(discountCurve);
               end
               
           else
               for i= idxTerm+1:length(couponRates)
                   out = out + couponRates{i}.BPS(discountCurve);
               end
           end
%             for i=1:length(floatingRateLeg.couponRates)
%                 out = out + floatingRateLeg.couponRates{i}.BPS(discountCurve);
%             end
        end
        
        function out = StartDate(floatingRateLeg)
            % fixedCoupon
            fieldNames = fieldnames(floatingRateLeg.couponRates{1});
            if any(strcmp(fieldNames,'accrualStartDate'))
                out = floatingRateLeg.couponRates{1}.accrualStartDate;
            else % simple Cashflow
                out = floatingRateLeg.couponRates{1}.paymentDate;
            end
             
        end
        
        function out = EndDate(floatingRateLeg)
            % fixedCoupon
            fieldNames = fieldnames(floatingRateLeg.couponRates{length(floatingRateLeg.couponRates)});
            if any(strcmp(fieldNames,'accrualEndDate'))
                out = floatingRateLeg.couponRates{length(floatingRateLeg.couponRates)}.accrualEndDate;
            else % simple Cashflow
                out = floatingRateLeg.couponRates{length(floatingRateLeg.couponRates)}.paymentDate;
            end
        end
        
    end
    
end



In [ ]:
classdef OvernightLegARRWithManualSchedule < H_RootObject
    % FixedRateLeg
    % properties: fixedSchedule 
    
    properties(SetAccess = public)
        nominals
        gearing
        spread
        overnightIndex
        manualSchedule
        liborFloatingSchedule
        calendar
        adjustingFactor
        paymentConvention
%         payLag
%         legSize
        couponRates
    end
    
    methods
        % constructor
        function floatingRateLeg = OvernightLegARRWithManualSchedule(manualSchedule,liborFloatingSchedule,overnightIndex,gearing,spread,nominal,...
                                      dayCounterLibor,dayCounter,paymentConvention,paymentCalendar,adjustingFactor)
                
                
                floatingRateLeg.manualSchedule = manualSchedule;
                floatingRateLeg.liborFloatingSchedule = liborFloatingSchedule;
                
                floatingRateLeg.calendar = paymentCalendar;
                floatingRateLeg.paymentConvention = paymentConvention;
                
                
                floatingRateLeg.overnightIndex = overnightIndex;
                
                floatingRateLeg.adjustingFactor = adjustingFactor;

                
                legSize = size(manualSchedule,1);
%                 legSize = length(schedule.dates)-1;
%                 floatingRateLeg.legSize = legSize;
                
                floatingRateLeg.spread = zeros(legSize,1);
                floatingRateLeg.nominals = zeros(legSize,1);
                
                for i=1:legSize
                    floatingRateLeg.nominals(i) = nominal;
                    floatingRateLeg.gearing(i) = gearing;
                    floatingRateLeg.spread(i) = spread;
                    
%                     startDate = schedule.dates{i};
%                     endDate = schedule.dates{i+1};
%                     paymentDate = floatingRateLeg.calendar.Advance(endDate,paymentConvention,...
%                                                                     payLag,'day',true);
                    
                    startDate   = manualSchedule{i,1};
                    endDate     = manualSchedule{i,2};
                    paymentDate = manualSchedule{i,3};
                    
                    refPeriodStartDate = liborFloatingSchedule{i,1};
                    refPeriodEndDate   = liborFloatingSchedule{i,2};
                    
                    
                    floatingRateLeg.couponRates{i} = OvernightIndexedCouponARR(paymentDate,nominal,overnightIndex,gearing,spread,dayCounterLibor,dayCounter,...
                                                     startDate,endDate,refPeriodStartDate,refPeriodEndDate,adjustingFactor);   
                end
                
        end
        
        function out = NPV(floatingRateLeg,discountCurve)
           out = 0;
            for i=1:length(floatingRateLeg.couponRates)
                out = out + floatingRateLeg.couponRates{i}.NPV(discountCurve);
            end
        end
        
        function out = BPS(floatingRateLeg,discountCurve)
           out = 0;
            for i=1:length(floatingRateLeg.couponRates)
                out = out + floatingRateLeg.couponRates{i}.BPS(discountCurve);
            end
        end
        
        function out = NPV_Term(floatingRateLeg,discountCurve,termDate,beforeAfterFlag)
           out = 0;
           idxTerm = 0;
           couponRates = floatingRateLeg.couponRates;
           % Find idxTerm
           % if beforeAfterFlag is before then
           % couponRates whose fixngDate is before(or equal to) termDate
           % are only compouted
           % -> to use the same function for overnightLeg
           % we change fixingDate to accrualStartDate
           % otherwise
           % couponRates whose fixingDate is
           for i=1:length(couponRates)
                if DateDiff(termDate,couponRates{i}.accrualStartDate) < 0
                    break;
                end
                idxTerm = idxTerm + 1;
           end
           
           if strcmp(beforeAfterFlag,'before')
               for i=1:idxTerm
                   out = out + couponRates{i}.NPV(discountCurve);
               end
               
           else
               for i= idxTerm+1:length(couponRates)
                   out = out + couponRates{i}.NPV(discountCurve);
               end
           end
           
%             for i=1:length(floatingRateLeg.couponRates)
%                 out = out + floatingRateLeg.couponRates{i}.NPV(discountCurve);
%             end

        end
        
        function out = BPS_Term(floatingRateLeg,discountCurve,termDate,beforeAfterFlag)
           out = 0;
           idxTerm = 0;
           couponRates = floatingRateLeg.couponRates;
           % Find idxTerm
           % if beforeAfterFlag is before then
           % couponRates whose fixngDate is before(or equal to) termDate
           % are only compouted
           % -> to use the same function for overnightLeg
           % we change fixingDate to accrualStartDate
           % otherwise
           % couponRates whose fixingDate is
           for i=1:length(couponRates)
                if DateDiff(termDate,couponRates{i}.accrualStartDate) < 0
                    break;
                end
                idxTerm = idxTerm + 1;
           end
           
           if strcmp(beforeAfterFlag,'before')
               for i=1:idxTerm
                   out = out + couponRates{i}.BPS(discountCurve);
               end
               
           else
               for i= idxTerm+1:length(couponRates)
                   out = out + couponRates{i}.BPS(discountCurve);
               end
           end
%             for i=1:length(floatingRateLeg.couponRates)
%                 out = out + floatingRateLeg.couponRates{i}.BPS(discountCurve);
%             end
        end
        
        function out = StartDate(floatingRateLeg)
            % fixedCoupon
            fieldNames = fieldnames(floatingRateLeg.couponRates{1});
            if any(strcmp(fieldNames,'accrualStartDate'))
                out = floatingRateLeg.couponRates{1}.accrualStartDate;
            else % simple Cashflow
                out = floatingRateLeg.couponRates{1}.paymentDate;
            end
             
        end
        
        function out = EndDate(floatingRateLeg)
            % fixedCoupon
            fieldNames = fieldnames(floatingRateLeg.couponRates{length(floatingRateLeg.couponRates)});
            if any(strcmp(fieldNames,'accrualEndDate'))
                out = floatingRateLeg.couponRates{length(floatingRateLeg.couponRates)}.accrualEndDate;
            else % simple Cashflow
                out = floatingRateLeg.couponRates{length(floatingRateLeg.couponRates)}.paymentDate;
            end
        end
        
    end
    
end



In [ ]:
classdef OvernightLeg < H_RootObject
    % FixedRateLeg
    % properties: fixedSchedule 
    
    properties(SetAccess = public)
        nominals
        gearing
        spread
        overnightIndex
        schedule
        calendar
        paymentConvention
        payLag
%         legSize
        couponRates
    end
    
    methods
        % constructor
        function floatingRateLeg = OvernightLeg(schedule,overnightIndex,gearing,spread,nominal,...
                                      dayCounter,paymentConvention,payLag)
                
                floatingRateLeg.schedule = schedule;
                floatingRateLeg.calendar = schedule.calendar;
                floatingRateLeg.paymentConvention = paymentConvention;
                floatingRateLeg.payLag = payLag;
                
                floatingRateLeg.overnightIndex = overnightIndex;
                
                
                
                legSize = length(schedule.dates)-1;
%                 floatingRateLeg.legSize = legSize;
                
                floatingRateLeg.spread = zeros(legSize,1);
                floatingRateLeg.nominals = zeros(legSize,1);
                
                for i=1:legSize
                    floatingRateLeg.nominals(i) = nominal;
                    floatingRateLeg.gearing(i) = gearing;
                    floatingRateLeg.spread(i) = spread;
                    startDate = schedule.dates{i};
                    endDate = schedule.dates{i+1};
                    
                    paymentDate = floatingRateLeg.calendar.Advance(endDate,paymentConvention,...
                                                                    payLag,'day',true);
%                     paymentDate = floatingRateLeg.calendar.Adjust(endDate,paymentConvention);
                    
                    floatingRateLeg.couponRates{i} = OvernightIndexedCoupon(paymentDate,nominal,overnightIndex,gearing,spread,dayCounter,...
                                                     startDate,endDate,startDate,endDate);   
                end
                
        end
        
        function out = NPV(floatingRateLeg,discountCurve)
           out = 0;
            for i=1:length(floatingRateLeg.couponRates)
                out = out + floatingRateLeg.couponRates{i}.NPV(discountCurve);
            end
        end
        
        function out = BPS(floatingRateLeg,discountCurve)
           out = 0;
            for i=1:length(floatingRateLeg.couponRates)
                out = out + floatingRateLeg.couponRates{i}.BPS(discountCurve);
            end
        end
        
        function out = NPV_Term(floatingRateLeg,discountCurve,termDate,beforeAfterFlag)
           out = 0;
           idxTerm = 0;
           couponRates = floatingRateLeg.couponRates;
           % Find idxTerm
           % if beforeAfterFlag is before then
           % couponRates whose fixngDate is before(or equal to) termDate
           % are only compouted
           % -> to use the same function for overnightLeg
           % we change fixingDate to accrualStartDate
           % otherwise
           % couponRates whose fixingDate is
           for i=1:length(couponRates)
                if DateDiff(termDate,couponRates{i}.accrualStartDate) < 0
                    break;
                end
                idxTerm = idxTerm + 1;
           end
           
           if strcmp(beforeAfterFlag,'before')
               for i=1:idxTerm
                   out = out + couponRates{i}.NPV(discountCurve);
               end
               
           else
               for i= idxTerm+1:length(couponRates)
                   out = out + couponRates{i}.NPV(discountCurve);
               end
           end
           
%             for i=1:length(floatingRateLeg.couponRates)
%                 out = out + floatingRateLeg.couponRates{i}.NPV(discountCurve);
%             end

        end
        
        function out = BPS_Term(floatingRateLeg,discountCurve,termDate,beforeAfterFlag)
           out = 0;
           idxTerm = 0;
           couponRates = floatingRateLeg.couponRates;
           % Find idxTerm
           % if beforeAfterFlag is before then
           % couponRates whose fixngDate is before(or equal to) termDate
           % are only compouted
           % -> to use the same function for overnightLeg
           % we change fixingDate to accrualStartDate
           % otherwise
           % couponRates whose fixingDate is
           for i=1:length(couponRates)
                if DateDiff(termDate,couponRates{i}.accrualStartDate) < 0
                    break;
                end
                idxTerm = idxTerm + 1;
           end
           
           if strcmp(beforeAfterFlag,'before')
               for i=1:idxTerm
                   out = out + couponRates{i}.BPS(discountCurve);
               end
               
           else
               for i= idxTerm+1:length(couponRates)
                   out = out + couponRates{i}.BPS(discountCurve);
               end
           end
%             for i=1:length(floatingRateLeg.couponRates)
%                 out = out + floatingRateLeg.couponRates{i}.BPS(discountCurve);
%             end
        end
        
        function out = StartDate(floatingRateLeg)
            % fixedCoupon
            fieldNames = fieldnames(floatingRateLeg.couponRates{1});
            if any(strcmp(fieldNames,'accrualStartDate'))
                out = floatingRateLeg.couponRates{1}.accrualStartDate;
            else % simple Cashflow
                out = floatingRateLeg.couponRates{1}.paymentDate;
            end
             
        end
        
        function out = EndDate(floatingRateLeg)
            % fixedCoupon
            fieldNames = fieldnames(floatingRateLeg.couponRates{length(floatingRateLeg.couponRates)});
            if any(strcmp(fieldNames,'accrualEndDate'))
                out = floatingRateLeg.couponRates{length(floatingRateLeg.couponRates)}.accrualEndDate;
            else % simple Cashflow
                out = floatingRateLeg.couponRates{length(floatingRateLeg.couponRates)}.paymentDate;
            end
        end
        
    end
    
end



In [ ]:
classdef OvernightIndexedCouponARR < H_RootObject
    % FixedRateLeg
    % properties: fixedSchedule 
    
    properties(SetAccess = public)
        paymentDate
        nominal
        accrualStartDate
        accrualEndDate
        refPeriodStartDate
        refPeriodEndDate
        
        overnightIndex
        gearing
        spread
        dayCounter
        dayCounterLibor
        
        valueDates
        fixingDates
        fixings
        dts
        
        % raw floating coupon is rounded by roundingNumber
        % SOFR : 7
        % SONIA: 6
        roundingNumber
        
        fixingValueDate
        fixingEndDate
        spanningTime
        adjustingFactor
    end
    
    methods
        % constructor
        function floatingRateCoupon = OvernightIndexedCouponARR(paymentDate,nominal,overnightIndex,gearing,spread,dayCounterLibor,dayCounter, ...
                                                   accrualStartDate,accrualEndDate,refPeriodStartDate,refPeriodEndDate,adjustingFactor)
            
            floatingRateCoupon.paymentDate = paymentDate;
            floatingRateCoupon.nominal = nominal;
            
            floatingRateCoupon.accrualStartDate = accrualStartDate;
            floatingRateCoupon.accrualEndDate = accrualEndDate;
            floatingRateCoupon.refPeriodStartDate = refPeriodStartDate;
            floatingRateCoupon.refPeriodEndDate = refPeriodEndDate;
            
            floatingRateCoupon.overnightIndex = overnightIndex;
            
            floatingRateCoupon.gearing = gearing;
            floatingRateCoupon.spread = spread;
            
            floatingRateCoupon.dayCounter = dayCounter;
            floatingRateCoupon.dayCounterLibor = dayCounterLibor;

            floatingRateCoupon.adjustingFactor = adjustingFactor;
            
            floatingRateCoupon.roundingNumber = 7;
            
            if overnightIndex.useSimpleTeleScoping
%                 overnightIndexCalcSchedule = SimpleSchedule(accrualStartDate,accrualEndDate,'1D' ,overnightIndex.fixingCalendar,...
%                                     overnightIndex.convention, 'Backward',false);
                                
                overnightIndexCalcSchedule = SimpleScheduleWTermConvention(accrualStartDate,accrualEndDate,'1D' ,overnightIndex.fixingCalendar,...
                                    overnightIndex.convention,'Unadjusted', 'Backward',false);
            else
                
%                 overnightIndexCalcSchedule = Schedule(accrualStartDate,accrualEndDate,'1D' ,overnightIndex.fixingCalendar,...
%                                     overnightIndex.convention, 'Backward',false);
%                                 
                overnightIndexCalcSchedule = ScheduleWTermConvention(accrualStartDate,accrualEndDate,'1D' ,overnightIndex.fixingCalendar,...
                                    overnightIndex.convention,'Unadjusted', 'Backward',false);
            end
            
            % preserve accrual start and end date as given
            if overnightIndexCalcSchedule.dates{1} ~= accrualStartDate
                overnightIndexCalcSchedule.dates{1} = accrualStartDate;
            end
            
            if overnightIndexCalcSchedule.dates{end} ~= accrualEndDate
                overnightIndexCalcSchedule.dates{end} = accrualEndDate;
            end

            floatingRateCoupon.valueDates = overnightIndexCalcSchedule.dates;
            
            n = length(floatingRateCoupon.valueDates) -1;
            
            floatingRateCoupon.fixingDates = {};
            if overnightIndex.fixingDays == 0
                for idx=1:n
                    if overnightIndex.fixingCalendar.IsHoliday(floatingRateCoupon.valueDates{idx})
                        floatingRateCoupon.fixingDates{end+1} = overnightIndex.fixingCalendar.Adjust(floatingRateCoupon.valueDates{idx},...
                                                                'Preceding');
                    else
                        floatingRateCoupon.fixingDates{end+1} = floatingRateCoupon.valueDates{idx};
                    end
                end
                
            else
                for idx=1:n
                    floatingRateCoupon.fixingDates{end+1} = overnightIndex.fixingCalendar.Advance(floatingRateCoupon.valueDates{idx},'Preceding',...
                                   -1*overnightIndex.fixingDays,'day', false);
                end
            end
            
            
            floatingRateCoupon.dts = [];
            for idx=1:n
                tempDt = DayCountFraction(floatingRateCoupon.valueDates{idx},floatingRateCoupon.valueDates{idx+1},overnightIndex.dayCounter);
                floatingRateCoupon.dts = [floatingRateCoupon.dts; tempDt]; 
            end
            
%             floatingRateCoupon.fixingDate = overnightIndex.fixingCalendar.Advance(accrualStartDate,'Preceding',...
%                                   -1*overnightIndex.fixingDays,'day', false);
%             
%             floatingRateCoupon.fixingValueDate = overnightIndex.fixingCalendar.Advance(floatingRateCoupon.fixingDate,'Following',...
%                                   overnightIndex.fixingDays,'day', false);
%             
%             floatingRateCoupon.fixingEndDate = overnightIndex.fixingCalendar.AdvanceTenor(floatingRateCoupon.fixingValueDate,overnightIndex.convention,...
%                                       overnightIndex.tenor,overnightIndex.endOfMonth);
%             
%             floatingRateCoupon.spanningTime = DayCountFraction(floatingRateCoupon.fixingValueDate,floatingRateCoupon.fixingEndDate,overnightIndex.dayCounter);
                                  
                              
        end
        
        function out = Rate(floatingRateCoupon)

            fixingDates = floatingRateCoupon.fixingDates;
            
            dts         = floatingRateCoupon.dts;
            n = length(dts);
            indexDum = 1;
            compoundingFactor = 1.0;
            
            overnightIndex = floatingRateCoupon.overnightIndex;
            
            asOfDate = overnightIndex.forwardCurve.asOfDate;
            
            % past fixing to compounding
            while (indexDum < n+1 && DateDiff(fixingDates{indexDum},asOfDate) < 0)
                if overnightIndex.fixingCalendar.IsHoliday(fixingDates{indexDum})
                    tempFixingDate = overnightIndex.fixingCalendar.Adjust(fixingDates{indexDum},...
                                                                'Preceding');
                    pastFixing = overnightIndex.Fixing(tempFixingDate);                                        
                else
                    pastFixing = overnightIndex.Fixing(fixingDates{indexDum});
                end
                
                compoundingFactor = compoundingFactor * (1.0 + pastFixing * dts(indexDum));
                indexDum = indexDum + 1;
            end
            
            % telescopic forecasting fixing
            if indexDum < n+1
                valueDates = floatingRateCoupon.valueDates;
                t1 = DateDiff(valueDates{indexDum},asOfDate)/365.0;
                t2 = DateDiff(valueDates{n+1},asOfDate)/365.0;
                startDiscount = overnightIndex.forwardCurve.DF(t1);
                endDiscount = overnightIndex.forwardCurve.DF(t2);
                compoundingFactor = compoundingFactor * startDiscount/endDiscount; 
                
            end
            
%             dcf = floatingRateCoupon.AccuralPeriod();
            % floating Rate is computed as (compoundingFactor -
            % 1)/dcf(compoundingSchedule.startDate,compoundingSchedule.endDate)
            % then rounded to the nearest  1e-5 percentage point( or 1e-7 in real rate) 
            dcf1 = DayCountFraction(floatingRateCoupon.valueDates{1},floatingRateCoupon.valueDates{n+1},overnightIndex.dayCounter);
            
            out = (compoundingFactor - 1.0)/dcf1;
            
            out = out * floatingRateCoupon.adjustingFactor;
                
            if overnightIndex.useRounding
                out = round(out,floatingRateCoupon.roundingNumber);
            end   
%             if DateDiff(floatingRateCoupon.fixingDate,floatingRateCoupon.overnightIndex.forwardCurve.asOfDate) > 0 || ...
%               (DateDiff(floatingRateCoupon.fixingDate,floatingRateCoupon.overnightIndex.forwardCurve.asOfDate) == 0 && ...
%               floatingRateCoupon.overnightIndex.forecastTodaysFixing)      
% %                 out = ForecastFixing(overnightIndex,fixingDate);
%                 out = ForecastFixingSimple(floatingRateCoupon.overnightIndex,floatingRateCoupon.fixingValueDate,floatingRateCoupon.fixingEndDate, ...
%                                             floatingRateCoupon.spanningTime);
%             else
%                 % find from fixing
%                 out = 0;
%                 error('unimplemented');
%             end
% %             out = floatingRateCoupon.overnightIndex.Fixing(floatingRateCoupon.fixingDate);
            
        end
        
        function out = Amount(floatingRateCoupon)
            floatingRate = floatingRateCoupon.Rate();
            dcf          = floatingRateCoupon.AccuralPeriod();
            out =  (floatingRate + floatingRateCoupon.spread(1)) * dcf;

        end
        
        function out = AccuralPeriod(floatingRateCoupon)
            
            % Fallback Rate :  ARR + Spread, Coupon =FallbackRate *
            % dcf(Libor)
            
            out = DayCountFraction(floatingRateCoupon.refPeriodStartDate,floatingRateCoupon.refPeriodEndDate,...
                                            floatingRateCoupon.dayCounterLibor);
                                        
%             out = DayCountFraction(floatingRateCoupon.accrualStartDate,floatingRateCoupon.accrualEndDate,...
%                                             floatingRateCoupon.dayCounter);
%             disp('not implemented');
        end
        
        function out = NPV(floatingRateCoupon,discountCurve)
            out = 0.0;
            
            t1 = DateDiff(floatingRateCoupon.paymentDate,discountCurve.params('asOfDate'))/365.0;
            if t1 > 0
                out = floatingRateCoupon.nominal * floatingRateCoupon.Amount()*discountCurve.DF(t1);
            else
                out = 0.0;
            end
        end
        
        function out = BPS(floatingRateCoupon,discountCurve)
            out = 0.0;
            
            t1 = DateDiff(floatingRateCoupon.paymentDate,discountCurve.params('asOfDate'))/365.0;
            if t1 > 0
                out = floatingRateCoupon.nominal * floatingRateCoupon.AccuralPeriod()*discountCurve.DF(t1);
            else
                out = 0.0;
            end
        end
        
    end
        
end



In [ ]:
classdef OvernightIndexedCoupon < H_RootObject
    % FixedRateLeg
    % properties: fixedSchedule 
    
    properties(SetAccess = public)
        paymentDate
        nominal
        accrualStartDate
        accrualEndDate
        refPeriodStartDate
        refPeriodEndDate
        
        overnightIndex
        gearing
        spread
        dayCounter
        
        valueDates
        fixingDates
        fixings
        dts
        
        % raw floating coupon is rounded by roundingNumber
        % SOFR : 7
        % SONIA: 6
        roundingNumber
        
        fixingValueDate
        fixingEndDate
        spanningTime
    end
    
    methods
        % constructor
        function floatingRateCoupon = OvernightIndexedCoupon(paymentDate,nominal,overnightIndex,gearing,spread,dayCounter, ...
                                                   accrualStartDate,accrualEndDate,refPeriodStartDate,refPeriodEndDate)
            
            floatingRateCoupon.paymentDate = paymentDate;
            floatingRateCoupon.nominal = nominal;
            
            floatingRateCoupon.accrualStartDate = accrualStartDate;
            floatingRateCoupon.accrualEndDate = accrualEndDate;
            floatingRateCoupon.refPeriodStartDate = refPeriodStartDate;
            floatingRateCoupon.refPeriodEndDate = refPeriodEndDate;
            
            floatingRateCoupon.overnightIndex = overnightIndex;
            
            floatingRateCoupon.gearing = gearing;
            floatingRateCoupon.spread = spread;
            
            floatingRateCoupon.dayCounter = dayCounter;
            
            
            floatingRateCoupon.roundingNumber = 7;
            
            if overnightIndex.useSimpleTeleScoping
%                 overnightIndexCalcSchedule = SimpleSchedule(accrualStartDate,accrualEndDate,'1D' ,overnightIndex.fixingCalendar,...
%                                     overnightIndex.convention, 'Backward',false);
                                
                overnightIndexCalcSchedule = SimpleScheduleWTermConvention(accrualStartDate,accrualEndDate,'1D' ,overnightIndex.fixingCalendar,...
                                    overnightIndex.convention,'Unadjusted', 'Backward',false);
            else
                
%                 overnightIndexCalcSchedule = Schedule(accrualStartDate,accrualEndDate,'1D' ,overnightIndex.fixingCalendar,...
%                                     overnightIndex.convention, 'Backward',false);
%                                 
                overnightIndexCalcSchedule = ScheduleWTermConvention(accrualStartDate,accrualEndDate,'1D' ,overnightIndex.fixingCalendar,...
                                    overnightIndex.convention,'Unadjusted', 'Backward',false);
            end
            
            % preserve accrual start and end date as given
            if overnightIndexCalcSchedule.dates{1} ~= accrualStartDate
                overnightIndexCalcSchedule.dates{1} = accrualStartDate;
            end
            
            if overnightIndexCalcSchedule.dates{end} ~= accrualEndDate
                overnightIndexCalcSchedule.dates{end} = accrualEndDate;
            end

            floatingRateCoupon.valueDates = overnightIndexCalcSchedule.dates;
            
            n = length(floatingRateCoupon.valueDates) -1;
            
            floatingRateCoupon.fixingDates = {};
            if overnightIndex.fixingDays == 0
                for idx=1:n
                    if overnightIndex.fixingCalendar.IsHoliday(floatingRateCoupon.valueDates{idx})
                        floatingRateCoupon.fixingDates{end+1} = overnightIndex.fixingCalendar.Adjust(floatingRateCoupon.valueDates{idx},...
                                                                'Preceding');
                    else
                        floatingRateCoupon.fixingDates{end+1} = floatingRateCoupon.valueDates{idx};
                    end
                end
                
            else
                for idx=1:n
                    floatingRateCoupon.fixingDates{end+1} = overnightIndex.fixingCalendar.Advance(floatingRateCoupon.valueDates{idx},'Preceding',...
                                   -1*overnightIndex.fixingDays,'day', false);
                end
            end
            
            
            floatingRateCoupon.dts = [];
            for idx=1:n
                tempDt = DayCountFraction(floatingRateCoupon.valueDates{idx},floatingRateCoupon.valueDates{idx+1},overnightIndex.dayCounter);
                floatingRateCoupon.dts = [floatingRateCoupon.dts; tempDt]; 
            end
            
%             floatingRateCoupon.fixingDate = overnightIndex.fixingCalendar.Advance(accrualStartDate,'Preceding',...
%                                   -1*overnightIndex.fixingDays,'day', false);
%             
%             floatingRateCoupon.fixingValueDate = overnightIndex.fixingCalendar.Advance(floatingRateCoupon.fixingDate,'Following',...
%                                   overnightIndex.fixingDays,'day', false);
%             
%             floatingRateCoupon.fixingEndDate = overnightIndex.fixingCalendar.AdvanceTenor(floatingRateCoupon.fixingValueDate,overnightIndex.convention,...
%                                       overnightIndex.tenor,overnightIndex.endOfMonth);
%             
%             floatingRateCoupon.spanningTime = DayCountFraction(floatingRateCoupon.fixingValueDate,floatingRateCoupon.fixingEndDate,overnightIndex.dayCounter);
                                  
                              
        end
        
        function out = Rate(floatingRateCoupon)

            fixingDates = floatingRateCoupon.fixingDates;
            
            dts         = floatingRateCoupon.dts;
            n = length(dts);
            indexDum = 1;
            compoundingFactor = 1.0;
            
            overnightIndex = floatingRateCoupon.overnightIndex;
            
            asOfDate = overnightIndex.forwardCurve.asOfDate;
            
            % past fixing to compounding
            while (indexDum < n+1 && DateDiff(fixingDates{indexDum},asOfDate) < 0)
                if overnightIndex.fixingCalendar.IsHoliday(fixingDates{indexDum})
                    tempFixingDate = overnightIndex.fixingCalendar.Adjust(fixingDates{indexDum},...
                                                                'Preceding');
                    pastFixing = overnightIndex.Fixing(tempFixingDate);                                        
                else
                    pastFixing = overnightIndex.Fixing(fixingDates{indexDum});
                end
                
                compoundingFactor = compoundingFactor * (1.0 + pastFixing * dts(indexDum));
                indexDum = indexDum + 1;
            end
            
            % telescopic forecasting fixing
            if indexDum < n+1
                valueDates = floatingRateCoupon.valueDates;
                t1 = DateDiff(valueDates{indexDum},asOfDate)/365.0;
                t2 = DateDiff(valueDates{n+1},asOfDate)/365.0;
                startDiscount = overnightIndex.forwardCurve.DF(t1);
                endDiscount = overnightIndex.forwardCurve.DF(t2);
                compoundingFactor = compoundingFactor * startDiscount/endDiscount; 
                
            end
            
%             dcf = floatingRateCoupon.AccuralPeriod();
            % floating Rate is computed as (compoundingFactor -
            % 1)/dcf(compoundingSchedule.startDate,compoundingSchedule.endDate)
            % then rounded to the nearest  1e-5 percentage point( or 1e-7 in real rate) 
            dcf1 = DayCountFraction(floatingRateCoupon.valueDates{1},floatingRateCoupon.valueDates{n+1},overnightIndex.dayCounter);
            
            out = (compoundingFactor - 1.0)/dcf1;
            
            if overnightIndex.useRounding
                out = round(out,floatingRateCoupon.roundingNumber);
            end   
%             if DateDiff(floatingRateCoupon.fixingDate,floatingRateCoupon.overnightIndex.forwardCurve.asOfDate) > 0 || ...
%               (DateDiff(floatingRateCoupon.fixingDate,floatingRateCoupon.overnightIndex.forwardCurve.asOfDate) == 0 && ...
%               floatingRateCoupon.overnightIndex.forecastTodaysFixing)      
% %                 out = ForecastFixing(overnightIndex,fixingDate);
%                 out = ForecastFixingSimple(floatingRateCoupon.overnightIndex,floatingRateCoupon.fixingValueDate,floatingRateCoupon.fixingEndDate, ...
%                                             floatingRateCoupon.spanningTime);
%             else
%                 % find from fixing
%                 out = 0;
%                 error('unimplemented');
%             end
% %             out = floatingRateCoupon.overnightIndex.Fixing(floatingRateCoupon.fixingDate);
            
        end
        
        function out = Amount(floatingRateCoupon)
            floatingRate = floatingRateCoupon.Rate();
            dcf          = floatingRateCoupon.AccuralPeriod();
            out =  floatingRate * dcf;

        end
        
        function out = AccuralPeriod(floatingRateCoupon)
            out = DayCountFraction(floatingRateCoupon.accrualStartDate,floatingRateCoupon.accrualEndDate,...
                                            floatingRateCoupon.dayCounter);
%             disp('not implemented');
        end
        
        function out = NPV(floatingRateCoupon,discountCurve)
            out = 0.0;
            
            t1 = DateDiff(floatingRateCoupon.paymentDate,discountCurve.params('asOfDate'))/365.0;
            if t1 > 0
                out = floatingRateCoupon.nominal * floatingRateCoupon.Amount()*discountCurve.DF(t1);
            else
                out = 0.0;
            end
        end
        
        function out = BPS(floatingRateCoupon,discountCurve)
            out = 0.0;
            
            t1 = DateDiff(floatingRateCoupon.paymentDate,discountCurve.params('asOfDate'))/365.0;
            if t1 > 0
                out = floatingRateCoupon.nominal * floatingRateCoupon.AccuralPeriod()*discountCurve.DF(t1);
            else
                out = 0.0;
            end
        end
        
    end
        
end



In [ ]:
function [Ainv] = OneStepGF(n,x,y,d,A)
% n :number of points in the grid
% x,y,d : matrix coefficients from huge decomposition
% A : original matrix from a,b,c
% Ainv : Green Function
% Qa : one cumulative distribution

    Ainv = zeros(n,n);
    for i=1:n
        Ainv(i,i) = x(i)*y(i)/A(i,i);
    end

    %first row dealt separately
    for j=2:n
        Ainv(1,j) = Ainv(1,j-1)*y(j)/y(j-1)*(-A(j-1,j)/A(j,j));
    end


    for i=2:n
        for j=i-1:-1:1
            Ainv(i,j) = Ainv(i,j+1)*x(j)/x(j+1)*(-A(j+1,j)/A(j,j));
        end

        for j=i+1:n
            Ainv(i,j) = Ainv(i,j-1)*y(j)/y(j-1)*(-A(j-1,j)/A(j,j));
        end

    end
end



In [ ]:
classdef OneFactorGaussianCopula < H_RootObject
    % FixedRateLeg
    % properties: fixedSchedule 
    
    properties(SetAccess = public)
        correlation
        maximum
        minimum
        numberOfSteps
        dm
    end
    
    methods
        % constructor
        function oneFactorGaussianCopula = OneFactorGaussianCopula(params)
            
            oneFactorGaussianCopula.correlation = params('correlation');
            
            if isKey(params,'maximum')
                oneFactorGaussianCopula.maximum = params('maximum');
            else
                oneFactorGaussianCopula.maximum = 5.0;
            end
            
            if isKey(params,'minimum')
                oneFactorGaussianCopula.minimum = params('minimum');
            else
                oneFactorGaussianCopula.minimum = -5.0;
            end
            
            if isKey(params,'numberOfSteps')
                oneFactorGaussianCopula.numberOfSteps = params('numberOfSteps');
            else
                oneFactorGaussianCopula.numberOfSteps = 50;
            end
            
            oneFactorGaussianCopula.dm = (oneFactorGaussianCopula.maximum - oneFactorGaussianCopula.minimum)/oneFactorGaussianCopula.numberOfSteps;
            
        end
        
        function out = m(oneFactorGaussianCopula,idx)
            minimum = oneFactorGaussianCopula.minimum;
            dm = oneFactorGaussianCopula.dm;
            out = minimum + dm * (idx-1) + dm/2.0;

        end
        
        function out = Densitydm(oneFactorGaussianCopula,idx)
            density = H_ndf(oneFactorGaussianCopula.m(idx));
            out = density * oneFactorGaussianCopula.dm;

        end
        
        function out = ConditionalProb(oneFactorGaussianCopula,defaultProb,m)
            
            if defaultProb < 1e-10
                out = 0.0;
                return;
            end
            
            c = oneFactorGaussianCopula.correlation;
            
            res = H_ncdf((H_nicdf(defaultProb) -sqrt(c) * m)/sqrt(1.0 - c));
            out = res;
            
        end
        
        function out = ConditionalProbList(oneFactorGaussianCopula,defaultProbList,m)
            out = zeros(length(defaultProbList),1);
            for i=1:length(defaultProbList)
                out(i) = oneFactorGaussianCopula.ConditionalProb(defaultProbList(i),m);
            end
        end
        
        function out = Integrate(oneFactorGaussianCopula,N,defaultProbList)
            
            avg = 0.0;
            for i=1:oneFactorGaussianCopula.numberOfSteps
                conditionalProbList = oneFactorGaussianCopula.ConditionalProbList(defaultProbList,oneFactorGaussianCopula.m(i));
                prob = oneFactorGaussianCopula.ProbabilityOfAtLeastNEvents(N,conditionalProbList);
                avg = avg + prob * oneFactorGaussianCopula.Densitydm(i);
            end
            
            out = avg;
        end
        
        function out = ProbabilityOfNEvents(oneFactorGaussianCopula,p)
            
            numOfList = length(p);
            
            % 0th default, 1st default,2nd,.. AllDefault Prob
            prob = zeros(numOfList+1,0); 
            prev = zeros(numOfList+1,0);
            prob(1) = 1.0;
            
            for j=1:numOfList
                prev = prob;
                prob(1) = prev(1)*(1.0 - p(j));
                
                for i=2:j
                    prob(i) = prev(i-1) * p(j) + prev(i) * (1.0 - p(j));
                end
                
                prob(j+1) = prev(j) * p(j);
            end
            
            out = prob;
        end
        
        function out = ProbabilityOfAtLeastNEvents(oneFactorGaussianCopula,N,conditionalProbList)
            
            probList = oneFactorGaussianCopula.ProbabilityOfNEvents(conditionalProbList);
            sum = 1.0;
            for j=1:N
                sum = sum - probList(j);
            end
            
            out = sum;
        end
        
    end
    
end



In [ ]:
classdef OISRateHelper_DS < H_RootObject
    % FixedRateLeg
    % properties: fixedSchedule 
    
    properties(SetAccess = public)
        
        rateType
        
        rate
        rateTenor
        
        fixedLegTenor
        fixedLegConvention
        fixedLegDayCounter
        
        floatingLegTenor
        floatingLegConvention
        floatingLegDayCounter
        
        overnightIndex
        evaluationDate
        
        vanillaSwap
        discountCurve
        
        earliestDate
        latestDate
        
        spotLag
        payLag
        
        calcCalendar
        
%         useRounding
        
    end
    
    methods
        % constructor
        function oisRateHelper = OISRateHelper_DS(params)
                if nargin > 0
                    oisRateHelper.rateType = 'OIS';
                    
                    oisRateHelper.calcCalendar = params('calcCalendar');
                    
                    if isKey(params,'spotLag')
                        oisRateHelper.spotLag = params('spotLag');
                    else
                        oisRateHelper.spotLag = 2; % default
                     end
                    
                    if isKey(params,'payLag')
                        oisRateHelper.payLag = params('payLag');
                    else
                        oisRateHelper.payLag = 2; % default
                    end
                    
%                     if isKey(params,'useRounding')
%                         oisRateHelper.useRounding = params('useRounding');
%                     else
%                         oisRateHelper.useRounding = false; % default
%                     end
                    
                    oisRateHelper.rate = params('rate');
                    oisRateHelper.rateTenor = params('rateTenor');
                    
                    oisRateHelper.fixedLegTenor = params('fixedLegTenor');
                    oisRateHelper.fixedLegConvention = params('fixedLegConvention');
                    oisRateHelper.fixedLegDayCounter = params('fixedLegDayCounter');
                                        
                    oisRateHelper.floatingLegTenor = params('floatingLegTenor');
                    oisRateHelper.floatingLegConvention = params('floatingLegConvention');
                    oisRateHelper.floatingLegDayCounter = params('floatingLegDayCounter');
                    
                    overnightIndex = params('overnightIndex');
                    oisRateHelper.overnightIndex = overnightIndex;
                    
                    oisRateHelper.evaluationDate = overnightIndex.forwardCurve.asOfDate;
                    
                    oisRateHelper.discountCurve = params('discountCurve');
                    
                    if DateDiff(oisRateHelper.discountCurve.asOfDate,overnightIndex.forwardCurve.asOfDate)~=0
                        error('forwardCurve asOfDate is different from discountCurve asOfDate')
                    end
                    
                    oisRateHelper.vanillaSwap = MakeVanillaSwap(oisRateHelper);
                    
                else
                    oisRateHelper.rate = 0.02;
                    oisRateHelper.overnightIndex = overnightIndex();
                end
                
                out = InitializeDates(oisRateHelper);
                
                oisRateHelper.earliestDate = out.earliestDate;
                oisRateHelper.latestDate = out.latestDate;
                
        end
        
        function out = MakeVanillaSwap(oisRateHelper)
            overnightIndex = oisRateHelper.overnightIndex;
            referenceDate = overnightIndex.forwardCurve.asOfDate;
            
            
            startDate = oisRateHelper.calcCalendar.Advance(referenceDate,overnightIndex.convention,...
                                                oisRateHelper.spotLag,'day',overnightIndex.endOfMonth);
                                            
            useEndOfMonth = false;
            if IsEndOfMonth(startDate)
               useEndOfMonth = true;
            end
            
%             startDate = overnightIndex.fixingCalendar.Advance(referenceDate,overnightIndex.convention,...
%                                                 oisRateHelper.spotLag,'day',overnightIndex.endOfMonth);
            
%             startDate = overnightIndex.fixingCalendar.Advance(referenceDate,overnightIndex.convention,...
%                                                 overnightIndex.fixingDays,'day',overnightIndex.endOfMonth);
%             
                                            
            endDate = AddTenor(startDate,oisRateHelper.rateTenor);
            % fixed & floating schedule 
%             fixedSchedule = Schedule(startDate,endDate,oisRateHelper.fixedLegTenor ,overnightIndex.fixingCalendar,...
%                                     oisRateHelper.fixedLegConvention, 'Backward',false); 
%             
%             floatingSchedule = Schedule(startDate,endDate,oisRateHelper.floatingLegTenor,overnightIndex.fixingCalendar,...
%                                     oisRateHelper.floatingLegConvention, 'Backward',false);
                                
%             fixedSchedule = Schedule(startDate,endDate,oisRateHelper.fixedLegTenor ,oisRateHelper.calcCalendar,...
%                                     oisRateHelper.fixedLegConvention, 'Backward',false); 
%             
%             floatingSchedule = Schedule(startDate,endDate,oisRateHelper.floatingLegTenor,oisRateHelper.calcCalendar,...
%                                     oisRateHelper.floatingLegConvention, 'Backward',false);

             fixedSchedule = ScheduleWTermConvention(startDate,endDate,oisRateHelper.fixedLegTenor ,oisRateHelper.calcCalendar,...
                                    oisRateHelper.fixedLegConvention,oisRateHelper.fixedLegConvention, 'Backward',useEndOfMonth); 
            
            floatingSchedule = ScheduleWTermConvention(startDate,endDate,oisRateHelper.floatingLegTenor,oisRateHelper.calcCalendar,...
                                    oisRateHelper.floatingLegConvention,oisRateHelper.fixedLegConvention, 'Backward',useEndOfMonth);
                                
             type = 'Payer';
%              nominal = 10000.0;
             nominal = 1.0;
             fixedRate = 0.0;
             fixedDayCounter = oisRateHelper.fixedLegDayCounter;
             floatingSpread = 0.0;
             floatingDayCounter = oisRateHelper.floatingLegDayCounter;
             payLag = oisRateHelper.payLag;
%              vanillaSwap = VanillaSwap(type,nominal,fixedSchedule,fixedRate,fixedDayCounter, ...
%                                       floatingSchedule,overnightIndex,floatingSpread,floatingDayCounter);
                                  
             vanillaSwap = VanillaOvernightIndexSwap(type,nominal,fixedSchedule,fixedRate,fixedDayCounter, ...
                                      floatingSchedule,overnightIndex,floatingSpread,floatingDayCounter,payLag);
                                  
             out = vanillaSwap;
        end
        
        function out = InitializeDates(oisRateHelper)
            
            %earliestDate
            earliestDate = oisRateHelper.vanillaSwap.StartDate();
            latestDate = oisRateHelper.vanillaSwap.EndDate();
            
            % use PillarDate As last paydate only when
            % useLastEndDateAsPillarDate is false
            if ~oisRateHelper.overnightIndex.useLastEndDateAsPillarDate
            
                latestReleventDate = oisRateHelper.vanillaSwap.floatingLeg.couponRates{end}.paymentDate; 
                % if the last floating Index fixingEndDate is after
                % latestDate(last accrualEndDate then we change
                % the lastestDate into floatEndValueDate

                if DateDiff(latestReleventDate,latestDate) > 0
                    latestDate = latestReleventDate;
                end
            end
            
            out.earliestDate = earliestDate;
            out.latestDate = latestDate;
           
        end
        
        function out = ImpliedQuote(oisRateHelper)
            outInfo = oisRateHelper.vanillaSwap.Calculate(oisRateHelper.discountCurve);
            out = outInfo.fairRate;
            
        end
        
        % bootstrapping
        function out = QuoteError(oisRateHelper)
            out = oisRateHelper.rate - oisRateHelper.ImpliedQuote();
        end
        
        function out = BootStrapError(oisRateHelper,guess,idx)
            oisRateHelper.Update(idx,guess);
            out = oisRateHelper.QuoteError();
        end
        
        function Update(oisRateHelper,idx,guess)
             % forwardCurve update
            forwardRawData = oisRateHelper.overnightIndex.forwardCurve.params('rawData');
            forwardRawData(idx,2) = guess;
            oisRateHelper.overnightIndex.forwardCurve.params('rawData')= forwardRawData;
            
            % for singleCurve framework this is redundant
            
            % Multicurve, Dual Curve Stripping we do not update discount
            % Curve
            % disabled
            
%             % discountCurve update
%             discountRawData = oisRateHelper.discountCurve.params('rawData');
%             discountRawData(idx,2) = guess;
%             oisRateHelper.discountCurve.params('rawData')= discountRawData;
            
        end
        
        
    end
    
end



In [ ]:
classdef OISRateHelper < H_RootObject
    % FixedRateLeg
    % properties: fixedSchedule 
    
    properties(SetAccess = public)
        
        rateType
        
        rate
        rateTenor
        
        fixedLegTenor
        fixedLegConvention
        fixedLegDayCounter
        
        floatingLegTenor
        floatingLegConvention
        floatingLegDayCounter
        
        overnightIndex
        evaluationDate
        
        vanillaSwap
        discountCurve
        
        earliestDate
        latestDate
        
        spotLag
        payLag
        
        calcCalendar
        
%         useRounding
        
    end
    
    methods
        % constructor
        function oisRateHelper = OISRateHelper(params)
                if nargin > 0
                    oisRateHelper.rateType = 'OIS';
                    
                    oisRateHelper.calcCalendar = params('calcCalendar');
                    
                     if isKey(params,'spotLag')
                        oisRateHelper.spotLag = params('spotLag');
                    else
                        oisRateHelper.spotLag = 2; % default
                     end
                    
                    if isKey(params,'payLag')
                        oisRateHelper.payLag = params('payLag');
                    else
                        oisRateHelper.payLag = 2; % default
                    end
                    
%                     if isKey(params,'useRounding')
%                         oisRateHelper.useRounding = params('useRounding');
%                     else
%                         oisRateHelper.useRounding = false; % default
%                     end
                    
                    oisRateHelper.rate = params('rate');
                    oisRateHelper.rateTenor = params('rateTenor');
                    
                    oisRateHelper.fixedLegTenor = params('fixedLegTenor');
                    oisRateHelper.fixedLegConvention = params('fixedLegConvention');
                    oisRateHelper.fixedLegDayCounter = params('fixedLegDayCounter');
                                        
                    oisRateHelper.floatingLegTenor = params('floatingLegTenor');
                    oisRateHelper.floatingLegConvention = params('floatingLegConvention');
                    oisRateHelper.floatingLegDayCounter = params('floatingLegDayCounter');
                    
                    overnightIndex = params('overnightIndex');
                    oisRateHelper.overnightIndex = overnightIndex;
                    
                    oisRateHelper.evaluationDate = overnightIndex.forwardCurve.asOfDate;
                    
                    oisRateHelper.discountCurve = params('discountCurve');
                    
                    if oisRateHelper.discountCurve.asOfDate ~= overnightIndex.forwardCurve.asOfDate
                        error('forwardCurve asOfDate is different from discountCurve asOfDate')
                    end
                    
                    oisRateHelper.vanillaSwap = MakeVanillaSwap(oisRateHelper);
                    
                else
                    oisRateHelper.rate = 0.02;
                    oisRateHelper.overnightIndex = overnightIndex();
                end
                
                out = InitializeDates(oisRateHelper);
                
                oisRateHelper.earliestDate = out.earliestDate;
                oisRateHelper.latestDate = out.latestDate;
                
        end
        
        function out = MakeVanillaSwap(oisRateHelper)
            overnightIndex = oisRateHelper.overnightIndex;
            referenceDate = overnightIndex.forwardCurve.asOfDate;
            
            
            startDate = oisRateHelper.calcCalendar.Advance(referenceDate,overnightIndex.convention,...
                                                oisRateHelper.spotLag,'day',overnightIndex.endOfMonth);
                                            
                                            
             useEndOfMonth = false;
             if IsEndOfMonth(startDate)
                 useEndOfMonth = true;
             end
            
%             startDate = overnightIndex.fixingCalendar.Advance(referenceDate,overnightIndex.convention,...
%                                                 oisRateHelper.spotLag,'day',overnightIndex.endOfMonth);
            
%             startDate = overnightIndex.fixingCalendar.Advance(referenceDate,overnightIndex.convention,...
%                                                 overnightIndex.fixingDays,'day',overnightIndex.endOfMonth);
%             
                                            
            endDate = AddTenor(startDate,oisRateHelper.rateTenor);
            % fixed & floating schedule 
%             fixedSchedule = Schedule(startDate,endDate,oisRateHelper.fixedLegTenor ,overnightIndex.fixingCalendar,...
%                                     oisRateHelper.fixedLegConvention, 'Backward',false); 
%             
%             floatingSchedule = Schedule(startDate,endDate,oisRateHelper.floatingLegTenor,overnightIndex.fixingCalendar,...
%                                     oisRateHelper.floatingLegConvention, 'Backward',false);
                                
%             fixedSchedule = Schedule(startDate,endDate,oisRateHelper.fixedLegTenor ,oisRateHelper.calcCalendar,...
%                                     oisRateHelper.fixedLegConvention, 'Backward',false); 
%             
%             floatingSchedule = Schedule(startDate,endDate,oisRateHelper.floatingLegTenor,oisRateHelper.calcCalendar,...
%                                     oisRateHelper.floatingLegConvention, 'Backward',false);
                                
%              fixedSchedule = Schedule(startDate,endDate,oisRateHelper.fixedLegTenor ,oisRateHelper.calcCalendar,...
%                                     oisRateHelper.fixedLegConvention, 'Backward',useEndOfMonth); 
%             
%             floatingSchedule = Schedule(startDate,endDate,oisRateHelper.floatingLegTenor,oisRateHelper.calcCalendar,...
%                                     oisRateHelper.floatingLegConvention, 'Backward',useEndOfMonth);
                                
            fixedSchedule = ScheduleWTermConvention(startDate,endDate,oisRateHelper.fixedLegTenor ,oisRateHelper.calcCalendar,...
                                    oisRateHelper.fixedLegConvention,oisRateHelper.fixedLegConvention, 'Backward',useEndOfMonth); 
            
            floatingSchedule = ScheduleWTermConvention(startDate,endDate,oisRateHelper.floatingLegTenor,oisRateHelper.calcCalendar,...
                                    oisRateHelper.floatingLegConvention,oisRateHelper.fixedLegConvention, 'Backward',useEndOfMonth);
             
             type = 'Payer';
%              nominal = 10000.0;
             nominal = 1.0;
             fixedRate = 0.0;
             fixedDayCounter = oisRateHelper.fixedLegDayCounter;
             floatingSpread = 0.0;
             floatingDayCounter = oisRateHelper.floatingLegDayCounter;
             payLag = oisRateHelper.payLag;
%              vanillaSwap = VanillaSwap(type,nominal,fixedSchedule,fixedRate,fixedDayCounter, ...
%                                       floatingSchedule,overnightIndex,floatingSpread,floatingDayCounter);
                                  
             vanillaSwap = VanillaOvernightIndexSwap(type,nominal,fixedSchedule,fixedRate,fixedDayCounter, ...
                                      floatingSchedule,overnightIndex,floatingSpread,floatingDayCounter,payLag);
                                  
             out = vanillaSwap;
        end
        
        function out = InitializeDates(oisRateHelper)
            
            %earliestDate
            earliestDate = oisRateHelper.vanillaSwap.StartDate();
            latestDate = oisRateHelper.vanillaSwap.EndDate();
            
            % use PillarDate As last paydate only when
            % useLastEndDateAsPillarDate is false
            if ~oisRateHelper.overnightIndex.useLastEndDateAsPillarDate
            
                latestReleventDate = oisRateHelper.vanillaSwap.floatingLeg.couponRates{end}.paymentDate; 
                % if the last floating Index fixingEndDate is after
                % latestDate(last accrualEndDate then we change
                % the lastestDate into floatEndValueDate

                if DateDiff(latestReleventDate,latestDate) > 0
                    latestDate = latestReleventDate;
                end
            end
            
            out.earliestDate = earliestDate;
            out.latestDate = latestDate;
           
        end
        
        function out = ImpliedQuote(oisRateHelper)
            outInfo = oisRateHelper.vanillaSwap.Calculate(oisRateHelper.discountCurve);
            out = outInfo.fairRate;
            
        end
        
        % bootstrapping
        function out = QuoteError(oisRateHelper)
            out = oisRateHelper.rate - oisRateHelper.ImpliedQuote();
        end
        
        function out = BootStrapError(oisRateHelper,guess,idx)
            oisRateHelper.Update(idx,guess);
            out = oisRateHelper.QuoteError();
        end
        
        function Update(oisRateHelper,idx,guess)
             % forwardCurve update
            forwardRawData = oisRateHelper.overnightIndex.forwardCurve.params('rawData');
            forwardRawData(idx,2) = guess;
            oisRateHelper.overnightIndex.forwardCurve.params('rawData')= forwardRawData;
            
            % for singleCurve framework this is redundant
            
            % discountCurve update
            discountRawData = oisRateHelper.discountCurve.params('rawData');
            discountRawData(idx,2) = guess;
            oisRateHelper.discountCurve.params('rawData')= discountRawData;
            
        end
        
        
    end
    
end



In [ ]:
function out = NVolFromLNVol(forward,strike,lnVol,expiry)
% forward,strike,nVol,expiry -> lnVol

if forward >= strike
    otmPrice = blackPrice(forward,strike,lnVol,expiry,-1);
    nVol = BisecNormIV('P',forward,strike,0,expiry,0.0001,100,otmPrice,1e-10,1000);
else
    otmPrice = blackPrice(forward,strike,lnVol,expiry,1);
    nVol = BisecNormIV('C',forward,strike,0,expiry,0.0001,100,otmPrice,1e-10,1000);
end

out = nVol;
end



In [ ]:
classdef NumMethodOld < H_RootObject
    %Pricer(Security,Model,NumMethod)
    %Prcer is constructed from Security, Model, NumMethod 
    
    properties
        numMethodType
        mcOneTimeStep
        pathSize
        regressionOrder
    end
    
    methods
        function numMethod = NumMethodOld(numMethodInfo)
                numMethod.numMethodType = numMethodInfo('numMethodType');
                numMethod.mcOneTimeStep = numMethodInfo('mcOneTimeStep');
                numMethod.pathSize = numMethodInfo('pathSize');
                if ~isKey(numMethodInfo,'regressionOrder')
                    numMethod.regressionOrder = 2;
                end
%                 numMethod.regressionOrder = numMethodInfo('regressionOrder');
        end
        
        function out = numMethodTimeStepsGenerateDWCallableSwap(numMethod,valueDate,...
                        schedule,callSchedule,floatingSchedule,mcOneTimeStep)
            
            % we first add all eventsTimes from schedule, callSchedule, and
            % floatingSchedule
            
            %     schedule : startDates & endDates
            % callSchedule : resetDates
            %floatSchedule : startDates
            
            allEventTime = [];
            
            couponEventTime = [];
            couponStartTime = [];
            couponEndTime = [];
            inclusiveYN = zeros(schedule.numOfPeriod,1);
            oneTimeStep = zeros(schedule.numOfPeriod,1);
            numOfAliveCoupon = 0;
            numOfExpiredCoupon = 0;
            for idx=1:schedule.numOfPeriod
                if DateDiff(valueDate,schedule.payDates(idx)) >= 0
                    numOfExpiredCoupon = numOfExpiredCoupon + 1;
                    continue;
                end
                numOfAliveCoupon = numOfAliveCoupon + 1;
                
                if DateDiff(valueDate,schedule.startDates(idx)) >= 0
                    couponStartTime = [couponStartTime;0;];
                    inclusiveYN(idx) = 1;
                    oneTimeStep(idx) = 1;
                else
                    couponStartTime = [couponStartTime;DateDiff(schedule.startDates(idx),valueDate)];
                    inclusiveYN(idx) = 0;
                    if numOfAliveCoupon == 2
                        oneTimeStep(idx) = 1;
                    else
                        oneTimeStep(idx) = mcOneTimeStep;
                    end
                end
                couponEndTime = [couponEndTime;DateDiff(schedule.endDates(idx),valueDate)];
            end
            
            lastAliveCouponIdx = 1 + numOfExpiredCoupon; 
            
            allEventTime = [couponStartTime;couponEndTime;allEventTime];
            % floatingSchedule
            floatingEventTime = [];
            numOfAliveFloatingCoupon = 0;
            numOfExpiredFloatingCoupon = 0;
            
            for idx=1:floatingSchedule.numOfPeriod
                if DateDiff(valueDate,floatingSchedule.payDates(idx)) >= 0
                    numOfExpiredFloatingCoupon = numOfExpiredFloatingCoupon + 1;
                    continue;
                end
                numOfAliveFloatingCoupon = numOfAliveFloatingCoupon + 1;
                
                if DateDiff(valueDate,floatingSchedule.startDates(idx)) >= 0
                    floatingEventTime = [floatingEventTime;0];
                else
                    floatingEventTime = [floatingEventTime;DateDiff(floatingSchedule.startDates(idx),valueDate)];
                end
            end
            lastAliveFloatingCouponIdx = 1 + numOfExpiredFloatingCoupon;
            allEventTime = [floatingEventTime;allEventTime];
            % callSchedule
            callEventTime = [];
            numOfAliveCall = 0;
            numOfExpiredCall = 0;
            lastAliveCallIdx = 0;
            for idx=1:callSchedule.numOfPeriod
                if DateDiff(valueDate,callSchedule.resetDates(idx)) >= 0
                    numOfExpiredCall = numOfExpiredCall + 1;
                    continue;
                end
                numOfAliveCall = numOfAliveCall + 1;
                
                if DateDiff(valueDate,callSchedule.resetDates(idx)) >= 0
                    callEventTime = [callEventTime;0];
                else
                    callEventTime = [callEventTime;DateDiff(callSchedule.resetDates(idx),valueDate)];
                end
            end
            lastAliveCallIdx = 1 + numOfExpiredCall;
            allEventTime = [callEventTime;allEventTime];
            
            allEventTime = sort(unique(allEventTime),'ascend');
            % allEventsTimes for the eventTime & we have
            % callEventTime,floatEventTime
            % couponEventTime
            
            % we generate modelTimeSteps
            % we first generate from coupon Schedule
            % and we add additional floatEventTime
            % and callEventTime later
            
            % first add daily weekly (structured) coupon steps
            periodCount = schedule.numOfPeriod;
            

            totalTimeSteps = [];
            totalTimeStepsSize = 0;
            numOfAlivePeriod = 0;
            
            for i=lastAliveCouponIdx:periodCount
                timeSteps = [];
                couponEndTimeP = couponEndTime(i);
                oneTimeStepP = oneTimeStep(i);
                
                timeSteps = [couponEndTimeP;timeSteps];
                
                % endTime-1 should be included
                if oneTimeStepP >  1
                    couponEndTimeP = couponEndTimeP - 1;
                    timeSteps = [couponEndTimeP;timeSteps];
                end
                
                while 1
                    timeSteps = [couponEndTimeP;timeSteps];
                    couponEndTimeP = couponEndTimeP - 1*oneTimeStepP;
                    if couponEndTimeP <= couponStartTime(i) break;end
                end

                % startTime should be included
                timeSteps = [couponStartTime(i);timeSteps];

                timeSteps = sort(unique(timeSteps),'ascend');
                totalTimeSteps = [totalTimeSteps;timeSteps];
            end
            
            totalTimeSteps = sort(unique(totalTimeSteps),'ascend');
            
            totalTimeSteps = [totalTimeSteps;floatingEventTime];
            totalTimeSteps = [totalTimeSteps;callEventTime];
            
            totalTimeSteps = sort(unique(totalTimeSteps),'ascend');
            totalTimeStepsSize = length(totalTimeSteps);
            % we have generated all the time steps now we label
            % eventTimeIdx
            couponStartTimeIdx = [];
            couponEndTimeIdx = [];
            floatingEventTimeIdx = [];
            callEventTimeIdx = [];
            
            for idx=lastAliveCouponIdx:schedule.numOfPeriod
                idx2 = find(totalTimeSteps >= couponStartTime(idx));
                if isempty(idx2)
                    disp('inconsistent schedule');
                else
                    couponStartTimeIdx(idx) = min(idx2);
                end
            end
            
            for idx=lastAliveCouponIdx:schedule.numOfPeriod
                idx2 = find(totalTimeSteps >= couponEndTime(idx));
                if isempty(idx2)
                    disp('inconsistent schedule');
                else
                    couponEndTimeIdx(idx) = min(idx2);
                end
            end
            
            % nominal event happen at the last couponEndTime
            % we treat this as separate event
            nominalEventTimeIdx = couponEndTimeIdx(end);
            nominalEventTime = couponEndTime(end);
            
            for idx=lastAliveCouponIdx:floatingSchedule.numOfPeriod
                idx2 = find(totalTimeSteps >= floatingEventTime(idx));
                if isempty(idx2)
                    disp('inconsistent schedule');
                else
                    floatingEventTimeIdx(idx) = min(idx2);
                end
            end
            
            for idx=lastAliveCouponIdx:callSchedule.numOfPeriod
                idx2 = find(totalTimeSteps >= callEventTime(idx));
                if isempty(idx2)
                    disp('inconsistent schedule');
                else
                    callEventTimeIdx(idx) = min(idx2);
                end
            end
            
            out.totalTimeSteps = totalTimeSteps;
            out.totalTimeStepsSize = totalTimeStepsSize;
            out.couponStartTimeIdx = couponStartTimeIdx';
            out.couponEndTimeIdx = couponEndTimeIdx';
            out.nominalEventTimeIdx = nominalEventTimeIdx;
            
            out.floatingEventTimeIdx = floatingEventTimeIdx';
            out.callEventTimeIdx = callEventTimeIdx';
            
            allEventTimeIdx = [couponStartTimeIdx';couponEndTimeIdx';floatingEventTimeIdx';callEventTimeIdx'];
            allEventTimeIdx = sort(unique(allEventTimeIdx),'ascend');
            out.allEventTimeIdx = allEventTimeIdx;
            
            out.inclusiveYN = inclusiveYN;
            out.oneTimeStep = oneTimeStep;
            
            out.lastAliveCouponIdx = lastAliveCouponIdx;
            out.lastAliveFloatingCouponIdx = lastAliveFloatingCouponIdx;
            out.lastAliveCallIdx = lastAliveCallIdx;
            
            out.couponStartTime = couponStartTime;
            out.couponEndTime = couponEndTime;
            out.nominalEventTime = nominalEventTime;
            
            out.floatingEventTime = floatingEventTime;
            out.callEventTime = callEventTime;
            
            out.allEventTime = allEventTime;
        end
        
        
        function out = numMethodTimeStepsGenerateDW(numMethod,valueDate,schedule,mcOneTimeStep)
            
            periodCount = schedule.numOfPeriod;
            out.timeStepsPerPeriod =  cell(periodCount,1);
            
            out.startTimeIdx = zeros(periodCount,1);
            out.endTimeIdx = zeros(periodCount,1);
            
            out.inclusiveYN = zeros(periodCount,1);
            out.freq = zeros(periodCount,1);
            out.numOfTimeSteps = zeros(periodCount,1);
            out.totalTimeSteps = [];
            out.totalTimeStepsSize = 0;
            numOfAlivePeriod = 0;
            
            for i=1:periodCount
                timeSteps = [];
                if DateDiff(valueDate,schedule.payDates(i)) >= 0
%                     out.timeStepsPerPeriod(i) = timeSteps;
                    continue;
                end
                
                numOfAlivePeriod = numOfAlivePeriod + 1;
                
                % we are in the middle of the first alive period
                % or before the first period start
                % in this periods steps are generated daily
                if (DateDiff(valueDate,schedule.startDates(i)) >= 0) || ...
                    (DateDiff(valueDate,schedule.startDates(i)) < 0 && i == 1)
                    % 1st period
                    % 1st period
                    out.inclusiveYN(i) =  1;
                    out.freq(i) = 1;
                    
                    if DateDiff(valueDate,schedule.startDates(i)) >= 0
                        startDate = valueDate;
                    else
                        startDate = schedule.startDates(i);
                    end
                    
                    endDate = schedule.endDates(i);
                    
                    timeSteps = [];
                    
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [endTime;timeSteps];
                        endDate = endDate.AddDate( -1 ,'day');
                        if endDate.DateDiff(startDate) <= 0 break;end
                    end
                    
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [startTime;timeSteps];
                    
                    timeSteps = sort(unique(timeSteps),'ascend');
                    %out.timeStepsPerPeriod {i,1}.timeSteps = [];
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps;timeSteps];
                    
                    out.startTimeIdx(i) = 1;
                    out.endTimeIdx(i) = length(timeSteps);
                    
                else
                    startDate = schedule.startDates(i);
                    endDate = schedule.endDates(i);
                    
                    timeSteps = [];
                    
                    out.inclusiveYN(i) =  0;
                    out.freq(i) = mcOneTimeStep;
                    
                    % if we are in the second period then we do daily range accral
                    % for the possible theta problem in the schedule
                    % end date
                    
                    if numOfAlivePeriod == 2 
                        oneTimeStep = 1;
                        out.freq(i) = 1;
                    
                    else
                        oneTimeStep = mcOneTimeStep;
                    end
                    
                    % we add endDate and endDate -1 day in the schedule
                    % (floating rate fixing)
                    
                    endTime = endDate.DateDiff(valueDate);
                    timeSteps = [endTime;timeSteps];
                    endDate = endDate.AddDate(-1,'day');
                        
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [endTime;timeSteps];
                        endDate = endDate.AddDate(-1*oneTimeStep,'day');
                        if endDate.DateDiff(startDate) <= 0 break;end
                    end
                    
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [startTime;timeSteps];
                    
                    timeSteps = sort(unique(timeSteps),'ascend');
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps;timeSteps];
                    
                    % itht period: ith period's startTime == (i-1)th period's endTime 
                    out.startTimeIdx(i) = out.endTimeIdx(i-1) + 1 -1;
                    out.endTimeIdx(i) = out.endTimeIdx(i-1) + length(timeSteps) -1;
                    
                end
            end
            
            % endDate of the last period (= Security's maturityDate should
            % be included
            
            endTime = DateDiff(schedule.endDates(periodCount),valueDate);
            out.totalTimeSteps = [out.totalTimeSteps;endTime];
            out.totalTimeSteps = sort(unique(out.totalTimeSteps),'ascend');
            out.totalTimeStepsSize = length(out.totalTimeSteps);
            
            out.periodCount = periodCount;
            out.numOfAlivePeriod = numOfAlivePeriod;
        end
        
        
        
        function out = numMethodTimeStepsGenerate(numMethod,valueDate,schedule,mcOneTimeStep)
            
            periodCount = schedule.numOfPeriod;
            out.timeStepsPerPeriod =  cell(periodCount,1);
            
            out.startTimeIdx = zeros(periodCount,1);
            out.endTimeIdx = zeros(periodCount,1);
            
            out.numOfTimeSteps = zeros(periodCount,1);
            out.totalTimeSteps = [];
            out.totalTimeStepsSize = 0;
            numOfAlivePeriod = 0;
            
            for i=1:periodCount
                timeSteps = [];
                if DateDiff(valueDate,schedule.payDates(i)) >= 0
                    out.timeStepsPerPeriod(i) = timeSteps;
                    continue;
                end
                
                numOfAlivePeriod = numOfAlivePeriod + 1;
                if DateDiff(valueDate,schedule.startDates(i)) >= 0
                    startDate = valueDate;
                    endDate = schedule.endDates(i);
                    % we exclude end date from the 
                    endDate = endDate.AddDate(-1,'day');
                    
                    timeSteps = [];
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [timeSteps; startTime];
                    
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [timeSteps; endTime];
                        endDate = endDate.AddDate(-1*mcOneTimeStep,'day');
                        if endDate.DateDiff(startDate) < 0 break;end
                    end
                    timeSteps = sort(unique(timeSteps),'ascend');
                    %out.timeStepsPerPeriod {i,1}.timeSteps = [];
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps; timeSteps];
                    
                    % 1st period
                    out.startTimeIdx(i) = 1;
                    out.endTimeIdx(i) = length(timeSteps);
                    
                else
                    startDate = schedule.startDates(i);
                    endDate = schedule.endDates(i);
                    % we exclude end date from the 
                    endDate = endDate.AddDate(-1,'day');
                    
                    timeSteps = [];
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [timeSteps; startTime];
                    
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [timeSteps; endTime];
                        
                        endDate = endDate.AddDate(-1*mcOneTimeStep,'day');
                        if endDate.DateDiff(startDate) < 0 break;end
                    end
                    timeSteps = sort(unique(timeSteps),'ascend');
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps; timeSteps];
                    
                    % 1st period
                    out.startTimeIdx(i) = out.endTimeIdx(i-1) + 1;
                    out.endTimeIdx(i) = out.endTimeIdx(i-1) + length(timeSteps);
                    
                end
            end
            
            % endDate of the last period (= Security's maturityDate should
            % be included
            
            endTime = DateDiff(schedule.endDates(periodCount),valueDate);
            out.totalTimeSteps = [out.totalTimeSteps;endTime];
            out.totalTimeSteps = sort(unique(out.totalTimeSteps),'ascend');
            out.totalTimeStepsSize = length(out.totalTimeSteps);
            
            out.periodCount = periodCount;
            out.numOfAlivePeriod = numOfAlivePeriod;
        end
        
        function out = generateModelStates(numMethod,valueDate,schedule,numMethodInfo,model)
            
            pathSize = numMethodInfo.pathSize;
            numOfFactors = numMethodInfo.numOfFactors;
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            totalTimeSteps = numMethodInfo.totalTimeSteps;
            Z = numMethodInfo.Z;
            modelStates = zeros(numOfFactors*totalTimeStepsSize,pathSize);
            
            currentStep = totalTimeSteps(1);
            for i=1:totalTimeStepsSize-1
                nextStep = totalTimeSteps(i+1);
                stateLocalVariance = model.stateLocalVariance(currentStep,nextStep);
                stateLocalDrift = model.stateLocalDrift(currentStep,nextStep);
                L = chol(stateLocalVariance,'lower');
                prevModelStates = modelStates(numOfFactors*(i-1)+1:numOfFactors*(i-1)+2,:);
                dW = Z(numOfFactors*i+1:numOfFactors*i+2,:);
                modelStates(numOfFactors*i+1:numOfFactors*i+2,:) = stateLocalDrift * prevModelStates + L*dW; 
                currentStep = nextStep;
            end
            
            out = modelStates;
        end
        
        function out = generateModelStates1(numMethod,valueDate,schedule,numMethodInfo,model)
            
            pathSize = numMethodInfo.pathSize;
            numOfFactors = numMethodInfo.numOfFactors;
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            totalTimeSteps = numMethodInfo.totalTimeSteps;
            Z = numMethodInfo.Z;
            modelStates = zeros(numOfFactors*totalTimeStepsSize,pathSize);
            
            currentStep = totalTimeSteps(1);
            for i=1:totalTimeStepsSize-1
                nextStep = totalTimeSteps(i+1);
                stateLocalVariance = model.stateLocalVariance(currentStep,nextStep);
                stateLocalDrift = model.stateLocalDrift(currentStep,nextStep);
                L = chol(stateLocalVariance,'lower');
                prevModelStates = modelStates(numOfFactors*(i-1)+1,:);
                dW = Z(numOfFactors*i+1,:);
                modelStates(numOfFactors*i+1,:) = stateLocalDrift * prevModelStates + L*dW; 
                currentStep = nextStep;
            end
            
            out = modelStates;
        end
        
        function out = generateModelStates3(numMethod,valueDate,schedule,numMethodInfo,model)
            
            pathSize = numMethodInfo.pathSize;
            numOfFactors = numMethodInfo.numOfFactors;
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            totalTimeSteps = numMethodInfo.totalTimeSteps;
            Z = numMethodInfo.Z;
            modelStates = zeros(numOfFactors*totalTimeStepsSize,pathSize);
            
            currentStep = totalTimeSteps(1);
            for i=1:totalTimeStepsSize-1
                nextStep = totalTimeSteps(i+1);
                stateLocalVariance = model.stateLocalVariance(currentStep,nextStep);
                stateLocalDrift = model.stateLocalDrift(currentStep,nextStep);
                L = chol(stateLocalVariance,'lower');
                prevModelStates = modelStates(numOfFactors*(i-1)+1:numOfFactors*(i-1)+3,:);
                dW = Z(numOfFactors*i+1:numOfFactors*i+3,:);
                modelStates(numOfFactors*i+1:numOfFactors*i+3,:) = stateLocalDrift * prevModelStates + L*dW; 
                currentStep = nextStep;
            end
            
            out = modelStates;
        end
        
        function out = generateModelStatesNF(numMethod,valueDate,schedule,numMethodInfo,model)
            
            pathSize = numMethodInfo.pathSize;
            numOfFactors = numMethodInfo.numOfFactors;
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            totalTimeSteps = numMethodInfo.totalTimeSteps;
            Z = numMethodInfo.Z;
            modelStates = zeros(numOfFactors*totalTimeStepsSize,pathSize);
            
            currentStep = totalTimeSteps(1);
            for i=1:totalTimeStepsSize-1
                nextStep = totalTimeSteps(i+1);
                stateLocalVariance = model.stateLocalVariance(currentStep,nextStep);
                stateLocalDrift = model.stateLocalDrift(currentStep,nextStep);
                L = chol(stateLocalVariance,'lower');
                prevModelStates = modelStates(numOfFactors*(i-1)+1:numOfFactors*(i-1)+numOfFactors,:);
                dW = Z(numOfFactors*i+1:numOfFactors*i+numOfFactors,:);
                modelStates(numOfFactors*i+1:numOfFactors*i+numOfFactors,:) = stateLocalDrift * prevModelStates + L*dW; 
                currentStep = nextStep;
            end
            
            out = modelStates;
        end
        
        function out = generateModelStatesNFOneTime(numMethod,valueDate,schedule,numMethodInfo,model)
            
            pathSize = numMethodInfo.pathSize;
            numOfFactors = numMethodInfo.numOfFactors;
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            totalTimeSteps = numMethodInfo.totalTimeSteps;
            Z = numMethodInfo.Z;
            
            pastTimeStepsSize = numMethodInfo.pastTimeStepsSize;
            % we generate random numbers for the past time steps
            % and throw away to use the same random numbers for the future time steps
            if pastTimeStepsSize > 0
                for i=1:pastTimeStepsSize
                    Z= randn(pathSize/2,numOfFactors*1);
                    Z=[Z;-Z];
                    Z=Z';
                end
            end
            
            modelStates = zeros(numOfFactors*totalTimeStepsSize,pathSize);
            
            currentStep = totalTimeSteps(1);
            for i=1:totalTimeStepsSize-1
                nextStep = totalTimeSteps(i+1);
                stateLocalVariance = model.stateLocalVariance(currentStep,nextStep);
                stateLocalDrift = model.stateLocalDrift(currentStep,nextStep);
                L = chol(stateLocalVariance,'lower');
                prevModelStates = modelStates(numOfFactors*(i-1)+1:numOfFactors*(i-1)+numOfFactors,:);
                
                Z= randn(pathSize/2,numOfFactors*1);
                Z=[Z;-Z];
                Z=Z';
                
                dW = Z(1:numOfFactors,:);
%                 dW = Z(numOfFactors*i+1:numOfFactors*i+numOfFactors,:);
                modelStates(numOfFactors*i+1:numOfFactors*i+numOfFactors,:) = stateLocalDrift * prevModelStates + L*dW; 
                currentStep = nextStep;
            end
            
            out = modelStates;
        end
        
        function out = generateModelStates_reUseRNContainer(numMethod,valueDate,schedule,numMethodInfo,model)
            
            pathSize = numMethodInfo.pathSize;
            numOfFactors = numMethodInfo.numOfFactors;
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            totalTimeSteps = numMethodInfo.totalTimeSteps;
            Z = numMethodInfo.Z;
            modelStates = zeros(numOfFactors*totalTimeStepsSize,pathSize);
            
            currentStep = totalTimeSteps(1);
            for i=1:totalTimeStepsSize-1
                nextStep = totalTimeSteps(i+1);
                stateLocalVariance = model.stateLocalVariance(currentStep,nextStep);
                L = chol(stateLocalVariance);
                prevModelStates = modelStates(numOfFactors*(i-1)+1:numOfFactors*(i-1)+2,:);
                dW = Z(numOfFactors*i+1:numOfFactors*i+2,:);
                modelStates(numOfFactors*i+1:numOfFactors*i+2,:) = prevModelStates + L*dW; 
                currentStep = nextStep;
            end
            
            out = modelStates;
        end
        
    end
    
end


In [ ]:
classdef NumMethod < H_RootObject
    %Pricer(Security,Model,NumMethod)
    %Prcer is constructed from Security, Model, NumMethod 
    
    properties
        numMethodType
        mcOneTimeStep
        pathSize
        regressionOrder
    end
    
    methods
        function numMethod = NumMethod(numMethodInfo)
                numMethod.numMethodType = numMethodInfo('numMethodType');
                numMethod.mcOneTimeStep = numMethodInfo('mcOneTimeStep');
                numMethod.pathSize = numMethodInfo('pathSize');
                if ~isKey(numMethodInfo,'regressionOrder')
                    numMethod.regressionOrder = 2;
                end
%                 numMethod.regressionOrder = numMethodInfo('regressionOrder');
        end
        
        function out = numMethodTimeStepsGenerateDWSwapSimpleCall(numMethod,valueDate,schedule,floatingSchedule,callSchedule,mcOneTimeStep)
            
            % existing daily weekly schedule 
            % from (structured) coupon schedule
%             structuredInfo = numMethodTimeStepsGenerateDW(numMethod,valueDate,schedule,mcOneTimeStep);
%             structuredInfo = numMethodTimeStepsGenerateDWGeneric(numMethod,valueDate,schedule,mcOneTimeStep);
            structuredAndFloatingInfo = numMethodTimeStepsGenerateDWSwapSimple(numMethod,valueDate,schedule,floatingSchedule,mcOneTimeStep);
            
            out = structuredAndFloatingInfo;
            
            % callSchedule added to timeSteps
            if ~isempty(fieldnames(callSchedule))
                % floatingEventTime from the floatingSchedule
                callEventTime = [];
                numOfAliveCallCoupon = 0;

                for idx=1:callSchedule.numOfPeriod
                    if DateDiff(valueDate,callSchedule.payDates(idx)) < 0
                        numOfAliveCallCoupon = numOfAliveCallCoupon + 1;
                    end

                    % callEvent happens at callStartDate !
                    % now we change call event happens at call ResetDate
    %                 callEventTime = [callEventTime;DateDiff(callSchedule.startDates(idx),valueDate)];
                    callEventTime = [callEventTime;DateDiff(callSchedule.resetDates(idx),valueDate)];
                end

                newTotalTimeSteps = [structuredAndFloatingInfo.totalTimeSteps;callEventTime];
                newTotalTimeSteps = sort(unique(newTotalTimeSteps),'ascend');

                out.totalTimeSteps = newTotalTimeSteps;
                out.totalTimeStepsSize = length(newTotalTimeSteps);

                % debugging crossChck
                out.totalTimeSteps2 = []; 
                out.totalTimeStepsSize2 = 0;

                periodCount  = schedule.numOfPeriod;
                startTimeIdx = structuredAndFloatingInfo.startTimeIdx;
                endTimeIdx   = structuredAndFloatingInfo.endTimeIdx;
                startTime = structuredAndFloatingInfo.totalTimeSteps(startTimeIdx);
                endTime = structuredAndFloatingInfo.totalTimeSteps(endTimeIdx);
                for i=1:periodCount
                    idx1 = find((callEventTime >= startTime(i)) & (callEventTime <= endTime(i)));
                    if ~isempty(idx1)
                        callTimeSteps = callEventTime(idx1);
                        oriTimeSteps = structuredAndFloatingInfo.timeStepsPerPeriod{i,1}.timeSteps;
                        newTimeSteps = [oriTimeSteps;callTimeSteps];
                        newTimeSteps = sort(unique(newTimeSteps),'ascend');
                        out.timeStepsPerPeriod{i,1}.timeSteps = newTimeSteps;
                        out.numOfTimeSteps(i) = length(newTimeSteps);
                    end
                    out.totalTimeSteps2 = [out.timeStepsPerPeriod{i,1}.timeSteps;out.totalTimeSteps2];
                end

                out.totalTimeSteps2 = sort(unique(out.totalTimeSteps2),'ascend');
                % if there is extra time steps pass the last structured end
                % date then we add at the end
                idxDum = find(out.totalTimeSteps > out.totalTimeSteps2(end));

                if ~ isempty(idxDum)
                    out.totalTimeSteps2 = [out.totalTimeSteps2;out.totalTimeSteps(idxDum)];
                end

                out.totalTimeSteps2 = sort(unique(out.totalTimeSteps2),'ascend');
                out.totalTimeStepsSize2  = length(out.totalTimeSteps2);
                if out.totalTimeStepsSize2 ~= out.totalTimeStepsSize
                    error('incosistent call schedule')
                end

                for i=1:periodCount
                    idx1 = find(startTime(i) == out.totalTimeSteps);
                    out.startTimeIdx(i) = idx1;
                    idx2 = find(endTime(i) == out.totalTimeSteps);
                    out.endTimeIdx(i) = idx2;
                end    

                out.callEventTime = callEventTime;
                out.numOfAliveCallCoupon = numOfAliveCallCoupon;

                % modelTimeStpesGeneration

                pastTimeStepsSize = 0;
                modelTimeSteps = [];
                for i=1:out.totalTimeStepsSize
                    if out.totalTimeSteps(i) < 0
                        pastTimeStepsSize = pastTimeStepsSize + 1;
                    else
                        modelTimeSteps = [modelTimeSteps;out.totalTimeSteps(i)];
                    end
                end

                out.modelTimeSteps = modelTimeSteps;
                out.modelTimeSteps = sort(unique(out.modelTimeSteps),'ascend');


                out.modelTimeStepsSize = length(modelTimeSteps);
                out.pastTimeStepsSize = pastTimeStepsSize;

                % if asOfDate is past all event date
                % then we add 0(asOfDate) into timeSteps for (dummy)modelState
                % 
                if out.modelTimeStepsSize == 0
                    out.modelTimeSteps = [0];
                    out.modelTimeStepsSize = 1;

                    out.totalTimeSteps = [0;out.totalTimeSteps2];
                    out.totalTimeSteps = sort(unique(out.totalTimeSteps),'ascend');
                    out.totalTimeStepsSize = length(out.totalTimeSteps);
                end
            end
            
        end
        
        function out = numMethodTimeStepsGenerateDWSwapSimple(numMethod,valueDate,schedule,floatingSchedule,mcOneTimeStep)
            
            % existing daily weekly schedule 
            % from (structured) coupon schedule
%             structuredInfo = numMethodTimeStepsGenerateDW(numMethod,valueDate,schedule,mcOneTimeStep);
            structuredInfo = numMethodTimeStepsGenerateDWGeneric(numMethod,valueDate,schedule,mcOneTimeStep);
            
            out = structuredInfo;
            
            % floatingSchedule added to timeSteps
            if ~isempty(fieldnames(floatingSchedule))
                
                % floatingEventTime from the floatingSchedule
                floatingEventTime = [];
                numOfAliveFloatingCoupon = 0;

                for idx=1:floatingSchedule.numOfPeriod
                    if DateDiff(valueDate,floatingSchedule.payDates(idx)) < 0
                        numOfAliveFloatingCoupon = numOfAliveFloatingCoupon + 1;
                    end

                    floatingEventTime = [floatingEventTime;DateDiff(floatingSchedule.startDates(idx),valueDate)];
                end

                newTotalTimeSteps = [structuredInfo.totalTimeSteps;floatingEventTime];
                newTotalTimeSteps = sort(unique(newTotalTimeSteps),'ascend');

                out.totalTimeSteps = newTotalTimeSteps;
                out.totalTimeStepsSize = length(newTotalTimeSteps);

                % debugging crossChck
                out.totalTimeSteps2 = []; 
                out.totalTimeStepsSize2 = 0;

                periodCount  = schedule.numOfPeriod;
                startTimeIdx = structuredInfo.startTimeIdx;
                endTimeIdx   = structuredInfo.endTimeIdx;
                startTime = structuredInfo.totalTimeSteps(startTimeIdx);
                endTime = structuredInfo.totalTimeSteps(endTimeIdx);
                for i=1:periodCount
                    idx1 = find((floatingEventTime >= startTime(i)) & (floatingEventTime <= endTime(i)));
                    if ~isempty(idx1)
                        floatingTimeSteps = floatingEventTime(idx1);
                        oriTimeSteps = structuredInfo.timeStepsPerPeriod{i,1}.timeSteps;
                        newTimeSteps = [oriTimeSteps;floatingTimeSteps];
                        newTimeSteps = sort(unique(newTimeSteps),'ascend');
                        out.timeStepsPerPeriod{i,1}.timeSteps = newTimeSteps;
                        out.numOfTimeSteps(i) = length(newTimeSteps);
                    end
                    out.totalTimeSteps2 = [out.timeStepsPerPeriod{i,1}.timeSteps;out.totalTimeSteps2];
                end

                out.totalTimeSteps2 = sort(unique(out.totalTimeSteps2),'ascend');
                % if there is extra time steps pass the last structured end
                % date then we add at the end
                idxDum = find(out.totalTimeSteps > out.totalTimeSteps2(end));

                if ~ isempty(idxDum)
                    out.totalTimeSteps2 = [out.totalTimeSteps2;out.totalTimeSteps(idxDum)];
                end

                out.totalTimeSteps2 = sort(unique(out.totalTimeSteps2),'ascend');
                out.totalTimeStepsSize2  = length(out.totalTimeSteps2);
                if out.totalTimeStepsSize2 ~= out.totalTimeStepsSize
                    disp(['out.totalTimeStepsSize:',out.totalTimeStepsSize,'!=','out.totalTimeStepsSize2',out.totalTimeStepsSize2])
                    error('incosistent floating schedule')

                end

                for i=1:periodCount
                    idx1 = find(startTime(i) == out.totalTimeSteps);
                    out.startTimeIdx(i) = idx1;
                    idx2 = find(endTime(i) == out.totalTimeSteps);
                    out.endTimeIdx(i) = idx2;
                end    

                out.floatingEventTime = floatingEventTime;
                out.numOfAliveFloatingCoupon = numOfAliveFloatingCoupon;

                % modelTimeStpesGeneration

                pastTimeStepsSize = 0;
                modelTimeSteps = [];
                for i=1:out.totalTimeStepsSize
                    if out.totalTimeSteps(i) < 0
                        pastTimeStepsSize = pastTimeStepsSize + 1;
                    else
                        modelTimeSteps = [modelTimeSteps;out.totalTimeSteps(i)];
                    end
                end

                out.modelTimeSteps = modelTimeSteps;
                out.modelTimeSteps = sort(unique(out.modelTimeSteps),'ascend');


                out.modelTimeStepsSize = length(modelTimeSteps);
                out.pastTimeStepsSize = pastTimeStepsSize;

                % if asOfDate is past all event date
                % then we add 0(asOfDate) into timeSteps for (dummy)modelState
                % 
                if out.modelTimeStepsSize == 0
                    out.modelTimeSteps = [0];
                    out.modelTimeStepsSize = 1;

                    out.totalTimeSteps = [0;out.totalTimeSteps];
                    out.totalTimeSteps = sort(unique(out.totalTimeSteps),'ascend');
                    out.totalTimeStepsSize = length(out.totalTimeSteps);
                end
            end
            
        end
        
        function out = numMethodTimeStepsGenerateDWGeneric(numMethod,valueDate,schedule,mcOneTimeStep)
            
            periodCount = schedule.numOfPeriod;
            out.timeStepsPerPeriod =  cell(periodCount,1);
            
            out.startTimeIdx = zeros(periodCount,1);
            out.endTimeIdx = zeros(periodCount,1);
            
            out.inclusiveYN = zeros(periodCount,1);
            out.freq = zeros(periodCount,1);
            out.numOfTimeSteps = zeros(periodCount,1);
            out.totalTimeSteps = [];
            out.totalTimeStepsSize = 0;
            numOfAlivePeriod = 0;
            
            for i=1:periodCount
                timeSteps = [];
%                 if DateDiff(valueDate,schedule.payDates(i)) >= 0
% %                     out.timeStepsPerPeriod(i) = timeSteps;
%                     continue;
%                 end
%               
                % if valueDate < payDates(i) then corresponding Period is
                % alive
                
                if DateDiff(valueDate,schedule.payDates(i)) < 0
                    numOfAlivePeriod = numOfAlivePeriod + 1;
                end
                
                % we are in the middle of period
                % or before the first period start
                % in this periods steps are generated daily
                if (DateDiff(valueDate,schedule.startDates(i)) >= 0) || ...
                    (DateDiff(valueDate,schedule.startDates(i)) < 0 && i == 1)
                    % 1st period
                    % 1st period
                    out.inclusiveYN(i) =  1;
                    out.freq(i) = 1;
%                     
%                     if DateDiff(valueDate,schedule.startDates(i)) >= 0
%                         startDate = valueDate;
%                     else
%                         startDate = schedule.startDates(i);
%                     end
                    
                    startDate = schedule.startDates(i);
                    endDate = schedule.endDates(i);
                    
                    timeSteps = [];
                    
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [endTime;timeSteps];
                        endDate = endDate.AddDate( -1 ,'day');
                        if endDate.DateDiff(startDate) < 0 break;end
                    end
                    
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [startTime;timeSteps];
                    
                    timeSteps = sort(unique(timeSteps),'ascend');
                    %out.timeStepsPerPeriod {i,1}.timeSteps = [];
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps;timeSteps];
                    
                    if i == 1
                        out.startTimeIdx(i) = 1;
                        out.endTimeIdx(i) = length(timeSteps);
                    else
                        out.startTimeIdx(i) = out.endTimeIdx(i-1) + 1 -1;
                        out.endTimeIdx(i) = out.endTimeIdx(i-1) + length(timeSteps) -1;
                    end
                    
                else
                    startDate = schedule.startDates(i);
                    endDate = schedule.endDates(i);
                    
                    timeSteps = [];
                    
                    out.inclusiveYN(i) =  0;
                    out.freq(i) = mcOneTimeStep;
                    
                    % if we are in the second period then we do daily range accral
                    % for the possible theta problem in the schedule
                    % end date
                    
                    if numOfAlivePeriod == 2 
                        oneTimeStep = 1;
                        out.freq(i) = 1;
                    
                    else
                        oneTimeStep = mcOneTimeStep;
                    end
                    
                    % we add endDate and endDate -1 day in the schedule
                    % (floating rate fixing)
                    
                    endTime = endDate.DateDiff(valueDate);
                    timeSteps = [endTime;timeSteps];
                    endDate = endDate.AddDate(-1,'day');
                        
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [endTime;timeSteps];
                        endDate = endDate.AddDate(-1*oneTimeStep,'day');
                        if endDate.DateDiff(startDate) < 0 break;end
                    end
                    
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [startTime;timeSteps];
                    
                    timeSteps = sort(unique(timeSteps),'ascend');
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps;timeSteps];
                    
                    % itht period: ith period's startTime == (i-1)th period's endTime 
                    out.startTimeIdx(i) = out.endTimeIdx(i-1) + 1 -1;
                    out.endTimeIdx(i) = out.endTimeIdx(i-1) + length(timeSteps) -1;
                    
                end
                
                
            end
            
            % endDate of the last period (= Security's maturityDate should
            % be included
            
            endTime = DateDiff(schedule.endDates(periodCount),valueDate);
            out.totalTimeSteps = [out.totalTimeSteps;endTime];
            out.totalTimeSteps = sort(unique(out.totalTimeSteps),'ascend');
            out.totalTimeStepsSize = length(out.totalTimeSteps);
            
            out.periodCount = periodCount;
            out.numOfAlivePeriod = numOfAlivePeriod;
            
            
            % pastTimeSteps & modelTimeStpes Generation 
            
            pastTimeStepsSize = 0;
            modelTimeSteps = [];
            for i=1:out.totalTimeStepsSize
                if out.totalTimeSteps(i) < 0
                    pastTimeStepsSize = pastTimeStepsSize + 1;
                else
                    modelTimeSteps = [modelTimeSteps;out.totalTimeSteps(i)];
                end
            end
            
            out.modelTimeSteps = modelTimeSteps;
            out.modelTimeSteps = sort(unique(out.modelTimeSteps),'ascend');
            
            
            out.modelTimeStepsSize = length(modelTimeSteps);
            out.pastTimeStepsSize = pastTimeStepsSize;
            
            % if asOfDate is past all event date
            % then we add 0(asOfDate) into timeSteps for (dummy)modelState
            % 
            if out.modelTimeStepsSize == 0
                out.modelTimeSteps = [0];
                out.modelTimeStepsSize = 1;
                
                out.totalTimeSteps = [0;out.totalTimeSteps];
                out.totalTimeSteps = sort(unique(out.totalTimeSteps),'ascend');
                out.totalTimeStepsSize = length(out.totalTimeSteps);
            end
            
        end
        
        function out = numMethodTimeStepsGenerateDW(numMethod,valueDate,schedule,mcOneTimeStep)
            
            periodCount = schedule.numOfPeriod;
            out.timeStepsPerPeriod =  cell(periodCount,1);
            
            out.startTimeIdx = zeros(periodCount,1);
            out.endTimeIdx = zeros(periodCount,1);
            
            out.inclusiveYN = zeros(periodCount,1);
            out.freq = zeros(periodCount,1);
            out.numOfTimeSteps = zeros(periodCount,1);
            out.totalTimeSteps = [];
            out.totalTimeStepsSize = 0;
            numOfAlivePeriod = 0;
            
            for i=1:periodCount
                timeSteps = [];
                if DateDiff(valueDate,schedule.payDates(i)) >= 0
%                     out.timeStepsPerPeriod(i) = timeSteps;
                    continue;
                end
                
                numOfAlivePeriod = numOfAlivePeriod + 1;
                
                % we are in the middle of the first alive period
                % or before the first period start
                % in this periods steps are generated daily
                if (DateDiff(valueDate,schedule.startDates(i)) >= 0) || ...
                    (DateDiff(valueDate,schedule.startDates(i)) < 0 && i == 1)
                    % 1st period
                    % 1st period
                    out.inclusiveYN(i) =  1;
                    out.freq(i) = 1;
                    
                    if DateDiff(valueDate,schedule.startDates(i)) >= 0
                        startDate = valueDate;
                    else
                        startDate = schedule.startDates(i);
                    end
                    
                    endDate = schedule.endDates(i);
                    
                    timeSteps = [];
                    
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [endTime;timeSteps];
                        endDate = endDate.AddDate( -1 ,'day');
                        if endDate.DateDiff(startDate) <= 0 break;end
                    end
                    
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [startTime;timeSteps];
                    
                    timeSteps = sort(unique(timeSteps),'ascend');
                    %out.timeStepsPerPeriod {i,1}.timeSteps = [];
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps;timeSteps];
                    
                    out.startTimeIdx(i) = 1;
                    out.endTimeIdx(i) = length(timeSteps);
                    
                else
                    startDate = schedule.startDates(i);
                    endDate = schedule.endDates(i);
                    
                    timeSteps = [];
                    
                    out.inclusiveYN(i) =  0;
                    out.freq(i) = mcOneTimeStep;
                    
                    % if we are in the second period then we do daily range accral
                    % for the possible theta problem in the schedule
                    % end date
                    
                    if numOfAlivePeriod == 2 
                        oneTimeStep = 1;
                        out.freq(i) = 1;
                    
                    else
                        oneTimeStep = mcOneTimeStep;
                    end
                    
                    % we add endDate and endDate -1 day in the schedule
                    % (floating rate fixing)
                    
                    endTime = endDate.DateDiff(valueDate);
                    timeSteps = [endTime;timeSteps];
                    endDate = endDate.AddDate(-1,'day');
                        
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [endTime;timeSteps];
                        endDate = endDate.AddDate(-1*oneTimeStep,'day');
                        if endDate.DateDiff(startDate) <= 0 break;end
                    end
                    
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [startTime;timeSteps];
                    
                    timeSteps = sort(unique(timeSteps),'ascend');
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps;timeSteps];
                    
                    % itht period: ith period's startTime == (i-1)th period's endTime 
                    out.startTimeIdx(i) = out.endTimeIdx(i-1) + 1 -1;
                    out.endTimeIdx(i) = out.endTimeIdx(i-1) + length(timeSteps) -1;
                    
                end
            end
            
            % endDate of the last period (= Security's maturityDate should
            % be included
            
            endTime = DateDiff(schedule.endDates(periodCount),valueDate);
            out.totalTimeSteps = [out.totalTimeSteps;endTime];
            out.totalTimeSteps = sort(unique(out.totalTimeSteps),'ascend');
            out.totalTimeStepsSize = length(out.totalTimeSteps);
            
            out.periodCount = periodCount;
            out.numOfAlivePeriod = numOfAlivePeriod;
        end
        
        
        
        function out = numMethodTimeStepsGenerate(numMethod,valueDate,schedule,mcOneTimeStep)
            
            periodCount = schedule.numOfPeriod;
            out.timeStepsPerPeriod =  cell(periodCount,1);
            
            out.startTimeIdx = zeros(periodCount,1);
            out.endTimeIdx = zeros(periodCount,1);
            
            out.numOfTimeSteps = zeros(periodCount,1);
            out.totalTimeSteps = [];
            out.totalTimeStepsSize = 0;
            numOfAlivePeriod = 0;
            
            for i=1:periodCount
                timeSteps = [];
                if DateDiff(valueDate,schedule.payDates(i)) >= 0
                    out.timeStepsPerPeriod(i) = timeSteps;
                    continue;
                end
                
                numOfAlivePeriod = numOfAlivePeriod + 1;
                if DateDiff(valueDate,schedule.startDates(i)) >= 0
                    startDate = valueDate;
                    endDate = schedule.endDates(i);
                    % we exclude end date from the 
                    endDate = endDate.AddDate(-1,'day');
                    
                    timeSteps = [];
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [timeSteps; startTime];
                    
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [timeSteps; endTime];
                        endDate = endDate.AddDate(-1*mcOneTimeStep,'day');
                        if endDate.DateDiff(startDate) < 0 break;end
                    end
                    timeSteps = sort(unique(timeSteps),'ascend');
                    %out.timeStepsPerPeriod {i,1}.timeSteps = [];
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps; timeSteps];
                    
                    % 1st period
                    out.startTimeIdx(i) = 1;
                    out.endTimeIdx(i) = length(timeSteps);
                    
                else
                    startDate = schedule.startDates(i);
                    endDate = schedule.endDates(i);
                    % we exclude end date from the 
                    endDate = endDate.AddDate(-1,'day');
                    
                    timeSteps = [];
                    % startDate should be included
                    startTime = startDate.DateDiff(valueDate);
                    timeSteps = [timeSteps; startTime];
                    
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [timeSteps; endTime];
                        
                        endDate = endDate.AddDate(-1*mcOneTimeStep,'day');
                        if endDate.DateDiff(startDate) < 0 break;end
                    end
                    timeSteps = sort(unique(timeSteps),'ascend');
                    out.timeStepsPerPeriod{i,1}.timeSteps = timeSteps;
                    out.numOfTimeSteps(i) = length(timeSteps);
                    out.totalTimeSteps = [out.totalTimeSteps; timeSteps];
                    
                    % 1st period
                    out.startTimeIdx(i) = out.endTimeIdx(i-1) + 1;
                    out.endTimeIdx(i) = out.endTimeIdx(i-1) + length(timeSteps);
                    
                end
            end
            
            % endDate of the last period (= Security's maturityDate should
            % be included
            
            endTime = DateDiff(schedule.endDates(periodCount),valueDate);
            out.totalTimeSteps = [out.totalTimeSteps;endTime];
            out.totalTimeSteps = sort(unique(out.totalTimeSteps),'ascend');
            out.totalTimeStepsSize = length(out.totalTimeSteps);
            
            out.periodCount = periodCount;
            out.numOfAlivePeriod = numOfAlivePeriod;
        end
        
        function out = generateModelStates(numMethod,valueDate,schedule,numMethodInfo,model)
            
            pathSize = numMethodInfo.pathSize;
            numOfFactors = numMethodInfo.numOfFactors;
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            totalTimeSteps = numMethodInfo.totalTimeSteps;
            Z = numMethodInfo.Z;
            modelStates = zeros(numOfFactors*totalTimeStepsSize,pathSize);
            
            currentStep = totalTimeSteps(1);
            for i=1:totalTimeStepsSize-1
                nextStep = totalTimeSteps(i+1);
                stateLocalVariance = model.stateLocalVariance(currentStep,nextStep);
                stateLocalDrift = model.stateLocalDrift(currentStep,nextStep);
                L = chol(stateLocalVariance,'lower');
                prevModelStates = modelStates(numOfFactors*(i-1)+1:numOfFactors*(i-1)+2,:);
                dW = Z(numOfFactors*i+1:numOfFactors*i+2,:);
                modelStates(numOfFactors*i+1:numOfFactors*i+2,:) = stateLocalDrift * prevModelStates + L*dW; 
                currentStep = nextStep;
            end
            
            out = modelStates;
        end
        
        function out = generateModelStates1(numMethod,valueDate,schedule,numMethodInfo,model)
            
            pathSize = numMethodInfo.pathSize;
            numOfFactors = numMethodInfo.numOfFactors;
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            totalTimeSteps = numMethodInfo.totalTimeSteps;
            Z = numMethodInfo.Z;
            modelStates = zeros(numOfFactors*totalTimeStepsSize,pathSize);
            
            currentStep = totalTimeSteps(1);
            for i=1:totalTimeStepsSize-1
                nextStep = totalTimeSteps(i+1);
                stateLocalVariance = model.stateLocalVariance(currentStep,nextStep);
                stateLocalDrift = model.stateLocalDrift(currentStep,nextStep);
                L = chol(stateLocalVariance,'lower');
                prevModelStates = modelStates(numOfFactors*(i-1)+1,:);
                dW = Z(numOfFactors*i+1,:);
                modelStates(numOfFactors*i+1,:) = stateLocalDrift * prevModelStates + L*dW; 
                currentStep = nextStep;
            end
            
            out = modelStates;
        end
        
        function out = generateModelStates3(numMethod,valueDate,schedule,numMethodInfo,model)
            
            pathSize = numMethodInfo.pathSize;
            numOfFactors = numMethodInfo.numOfFactors;
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            totalTimeSteps = numMethodInfo.totalTimeSteps;
            Z = numMethodInfo.Z;
            modelStates = zeros(numOfFactors*totalTimeStepsSize,pathSize);
            
            currentStep = totalTimeSteps(1);
            for i=1:totalTimeStepsSize-1
                nextStep = totalTimeSteps(i+1);
                stateLocalVariance = model.stateLocalVariance(currentStep,nextStep);
                stateLocalDrift = model.stateLocalDrift(currentStep,nextStep);
                L = chol(stateLocalVariance,'lower');
                prevModelStates = modelStates(numOfFactors*(i-1)+1:numOfFactors*(i-1)+3,:);
                dW = Z(numOfFactors*i+1:numOfFactors*i+3,:);
                modelStates(numOfFactors*i+1:numOfFactors*i+3,:) = stateLocalDrift * prevModelStates + L*dW; 
                currentStep = nextStep;
            end
            
            out = modelStates;
        end
        
        function out = generateModelStatesNFNew(numMethod,valueDate,schedule,numMethodInfo,model)
            
            pathSize = numMethodInfo.pathSize;
            numOfFactors = numMethodInfo.numOfFactors;
            totalTimeStepsSize = numMethodInfo.modelTimeStepsSize;
            totalTimeSteps = numMethodInfo.modelTimeSteps;
            Z = numMethodInfo.Z;
            modelStates = zeros(numOfFactors*totalTimeStepsSize,pathSize);
            
            currentStep = totalTimeSteps(1);
            for i=1:totalTimeStepsSize-1
                nextStep = totalTimeSteps(i+1);
                stateLocalVariance = model.stateLocalVariance(currentStep,nextStep);
                stateLocalDrift = model.stateLocalDrift(currentStep,nextStep);
                L = chol(stateLocalVariance,'lower');
                prevModelStates = modelStates(numOfFactors*(i-1)+1:numOfFactors*(i-1)+numOfFactors,:);
                dW = Z(numOfFactors*i+1:numOfFactors*i+numOfFactors,:);
                modelStates(numOfFactors*i+1:numOfFactors*i+numOfFactors,:) = stateLocalDrift * prevModelStates + L*dW; 
                currentStep = nextStep;
            end
            
            out = modelStates;
        end
        
        function out = generateModelStatesNF(numMethod,valueDate,schedule,numMethodInfo,model)
            
            pathSize = numMethodInfo.pathSize;
            numOfFactors = numMethodInfo.numOfFactors;
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            totalTimeSteps = numMethodInfo.totalTimeSteps;
            Z = numMethodInfo.Z;
            modelStates = zeros(numOfFactors*totalTimeStepsSize,pathSize);
            
            currentStep = totalTimeSteps(1);
            for i=1:totalTimeStepsSize-1
                nextStep = totalTimeSteps(i+1);
                stateLocalVariance = model.stateLocalVariance(currentStep,nextStep);
                stateLocalDrift = model.stateLocalDrift(currentStep,nextStep);
                L = chol(stateLocalVariance,'lower');
%                 L = chol_mod(stateLocalVariance,'lower');
                prevModelStates = modelStates(numOfFactors*(i-1)+1:numOfFactors*(i-1)+numOfFactors,:);
                dW = Z(numOfFactors*i+1:numOfFactors*i+numOfFactors,:);
                modelStates(numOfFactors*i+1:numOfFactors*i+numOfFactors,:) = stateLocalDrift * prevModelStates + L*dW; 
                currentStep = nextStep;
            end
            
            out = modelStates;
        end
        
        function out = generateModelStatesNFHyb(numMethod,valueDate,schedule,numMethodInfo,model)
            
            numMatTime = numMethodInfo.numMatTime;
            pathSize = numMethodInfo.pathSize;
            numOfFactors = numMethodInfo.numOfFactors;
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            totalTimeSteps = numMethodInfo.totalTimeSteps;
            Z = numMethodInfo.Z;
            
             % initial modelState
           % IR 1 Factor & EQ 1 Factor & 1 Auxillary Factor
           %(drift as in cheyette model)
           
%             modelStates = zeros(numOfFactors*totalTimeStepsSize,pathSize);
            
            modelStates = zeros((numOfFactors+1)*totalTimeStepsSize,pathSize);
            
            currentStep = totalTimeSteps(1);
            for i=1:totalTimeStepsSize-1
                nextStep = totalTimeSteps(i+1);
                prevModelStates = modelStates((numOfFactors+1)*(i-1)+1:(numOfFactors+1)*(i-1)+(numOfFactors+1),:);
                dW = Z(numOfFactors*i+1:numOfFactors*i+numOfFactors,:);
                nextX = prevModelStates;
                mcmcOut   = model.inductMCForwardNew(currentStep,nextStep,numMatTime,nextX,dW);
                modelStates((numOfFactors+1)*i+1:(numOfFactors+1)*i+(numOfFactors+1),:) = mcmcOut.nextX;     
                currentStep = nextStep;
            end
            
            out = modelStates;
        end
        
        function out = generateModelStatesNFOneTime(numMethod,valueDate,schedule,numMethodInfo,model)
            
            pathSize = numMethodInfo.pathSize;
            numOfFactors = numMethodInfo.numOfFactors;
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            totalTimeSteps = numMethodInfo.totalTimeSteps;
            Z = numMethodInfo.Z;
            
            pastTimeStepsSize = numMethodInfo.pastTimeStepsSize;
            % we generate random numbers for the past time steps
            % and throw away to use the same random numbers for the future time steps
            if pastTimeStepsSize > 0
                for i=1:pastTimeStepsSize
                    Z= randn(pathSize/2,numOfFactors*1);
                    Z=[Z;-Z];
                    Z=Z';
                end
            end
            
            modelStates = zeros(numOfFactors*totalTimeStepsSize,pathSize);
            
            currentStep = totalTimeSteps(1);
            for i=1:totalTimeStepsSize-1
                nextStep = totalTimeSteps(i+1);
                stateLocalVariance = model.stateLocalVariance(currentStep,nextStep);
                stateLocalDrift = model.stateLocalDrift(currentStep,nextStep);
                L = chol(stateLocalVariance,'lower');
                prevModelStates = modelStates(numOfFactors*(i-1)+1:numOfFactors*(i-1)+numOfFactors,:);
                
                Z= randn(pathSize/2,numOfFactors*1);
                Z=[Z;-Z];
                Z=Z';
                
                dW = Z(1:numOfFactors,:);
%                 dW = Z(numOfFactors*i+1:numOfFactors*i+numOfFactors,:);
                modelStates(numOfFactors*i+1:numOfFactors*i+numOfFactors,:) = stateLocalDrift * prevModelStates + L*dW; 
                currentStep = nextStep;
            end
            
            out = modelStates;
        end
        
        function out = generateModelStates_reUseRNContainer(numMethod,valueDate,schedule,numMethodInfo,model)
            
            pathSize = numMethodInfo.pathSize;
            numOfFactors = numMethodInfo.numOfFactors;
            totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
            totalTimeSteps = numMethodInfo.totalTimeSteps;
            Z = numMethodInfo.Z;
            modelStates = zeros(numOfFactors*totalTimeStepsSize,pathSize);
            
            currentStep = totalTimeSteps(1);
            for i=1:totalTimeStepsSize-1
                nextStep = totalTimeSteps(i+1);
                stateLocalVariance = model.stateLocalVariance(currentStep,nextStep);
                L = chol(stateLocalVariance);
                prevModelStates = modelStates(numOfFactors*(i-1)+1:numOfFactors*(i-1)+2,:);
                dW = Z(numOfFactors*i+1:numOfFactors*i+2,:);
                modelStates(numOfFactors*i+1:numOfFactors*i+2,:) = prevModelStates + L*dW; 
                currentStep = nextStep;
            end
            
            out = modelStates;
        end
        
    end
    
end


In [ ]:
function [price] = NumericalIntegrationRoughHestonPrice(s_0,v_0,alpha,lambda,...
                                                v_bar,xi,rho,call,K,T,...
                                                varargin)
% Description: Computes prices of European call and put options under the
% rough Heston model as presented in e.g. (Gerhold et al., 2019) or (El
% Euch & Rosenbaum, 2018) and (El Euch & Rosenbaum 2019).
%
% The model for the asset price S(t) can be stated as
%
%   dS(t) = S(t)*sqrt(V(t))*dW_1(t)
%   V(t) = V(0) + int_0^t K(t-s) lambda*(v_bar - V(s))ds 
%               + int_0^t K(t-s) xi * sqrt(V(s))dW_2(s)
%
% with dW_1(t)dW_2(t) = rho dt and K(t) = (1/gamma(alpha))*t^(alpha-1).
% 
% We assume 1/2 < alpha <= 1 and V(0), lambda, v_bar, xi > 0 and 
% -1 <= rho < 0.
%
% To compute prices we use numerical integration via a Fourier transform
% as presented in e.g. (Gerhold et al., 2019) with an optimal integration 
% contour as proposed in (Lord & Kahl, 2006). To compute the characteristic
% function for the integration we solve a Volterra integral equation which 
% appears in e.g. (Gerhold et al., 2019) using the scheme from 
% (Diethelm, 2004).
%
% Parameters:
%   s_0:        [1x1 real] Initial value of asset price.
%   v_0:        [1x1 real] Initial value of variance process.
%   alpha:      [1x1 real] See equations above.
%   lambda:     [1x1 real] See equations above.
%   v_bar:      [1x1 real] See equations above.
%   xi:         [1x1 real] See equations above.
%   rho:        [1x1 real] See equations above.
%   call:       [1x1 boolean] true (call option), false (put option).
%   K:          [Nx1 real] Strike price(s).
%   T:          [Mx1 real] Expiration(s).
%   r:          [1x1 real] Risk-free interest rate.
%   q:          [1x1 real] Dividend yield.
%   beta:       [1x1 real, optional (default = see text)] Parameter for 
%               shifting the integration contour ideally to lower the 
%               oscillations of the integrand. Must be s.t. the beta'th 
%               moment of S(T) is finite (user responsibility). Default is 
%               an optimal value computed along the lines of 
%               (Lord & Kahl, 2006).
%   N:          [1x1 integer, optional(default=250)] Number of steps used 
%               to solve the Volterra integral equation.
%   ubound:     [1x1 real, optional(default=2000)] Upper truncation of 
%               Fourier integral.
%   midpoint:   [1x1 real, optional(default=10)] Used for multi-domain 
%               integration, see function 'NumericalIntegrationCall' and 
%               (Zhu, 2010).
%   dA:         [1x1 real, optional(default=50)] Same as above.
%   eps:        [1x1 real, optional(default=10^(-6))] Same as above.
%   nmax:       [1x1 integer, optional(default=200)] Number of terms in 
%               recursive scheme to compute moment explosion times which 
%               are needed when searching for an optimal contour 
%               shift, i.e. an optimal 'beta'.
%   disp_iter:  [1x1 boolean, optional(false)] If true and multiple prices 
%               are requested we show the time elapsed of each iteration.
% 
% Output: 
%   price: [NxM real] Prices of call or put options.
%   iv:    [NxM real] Black-scholes implied volatilities.
%
% Example: [price, iv] = NumericalIntegrationRoughHeston(100,0.2^2,0.6,2,...
%                                                  0.2^2,0.2,-0.8,true,...
%                                                  [90;100;110],[0.5;1])
%
% References:
%   - Stefan Gerhold, Christoph Gerstenecker, Arpad Pinter, Moment
%   explosions in the rough Heston model, Decisions in Economics and
%   Finance (2019) 42:575-608.
%
%   - Omar El Euch and Mathieu Rosenbaum, Perfect hedging in rough Heston
%   models, Mathematical Finance 28(6), 3813-3856, 2018.
%
%   - Omar El Euch and Mathieu Rosenbaum, The characteristic function of
%   rough Heston models, Mathematical Finance 29(1), 3-38, 2019.
%
%   - Roger lord and Christian Kahl, Optimal Fourier inversion in 
%   semi-analytical option pricing, Tinbergen Institute Discussion Paper 
%   No. 2006-066/2. Available at SRRN:
%   https://papers.ssrn.com/sol3/papers.cfm?abstract_id=921336.
% 
%   - Kai Diethelm, Neville J. Ford and Alan D. Freed, Detailed error
%   analysis for a fractional ADAMs method, Numerical Algorithms 36:31-52,
%   2004.
%

% Parse and check inputs:
p = inputParser;
addOptional(p,'r',0);
addOptional(p,'q',0);
addOptional(p,'beta',[]);
addOptional(p,'N',250);
addOptional(p,'ubound',2000);
addOptional(p,'midpoint',10);
addOptional(p,'dA',50);
addOptional(p,'eps',10^(-6));
addOptional(p,'nmax',200);
addOptional(p,'disp_iter',false);
parse(p,varargin{:});
r = p.Results.r;
q = p.Results.q;
beta = p.Results.beta;
N = p.Results.N;
ubound = p.Results.ubound;
midpoint = p.Results.midpoint;
dA = p.Results.dA;
eps = p.Results.eps;
nmax = p.Results.nmax;
disp_iter = p.Results.disp_iter;

if rho >= 0
    error('NumericalIntegrationRoughHeston: rho >= 0 is current not supported.');
end

if alpha > 1 || alpha <= 1/2
    error('NumericalIntegrationRoughHeston: Alpha must lie in (1/2,1].');
end

if any([v_0,lambda,v_bar,xi]) <= 0
    error(['NumericalIntegrationRoughHeston: v_0, lambda, v_bar and xi ',...
           'must be strictly positive.']);
end


if size(K,2) > 1 || size(T,2) > 1
    error(['NumericalIntegrationRoughHeston: Strike and expiration vectors ', ...
        'must be column vectors.']);
end

% Define characteristic function:
phi = @(u)(MomentGeneratingFunctionRoughHeston(v_0,alpha,lambda,v_bar,...
                                               xi,rho,T,1i*u,N));

% Vectorize by recursive calls:
tot_iter = numel(K)*numel(T);
if tot_iter > 1
    [price,iv] = deal(NaN(size(K,1),size(T,1)));
    ii = 1;
    for i=1:numel(K)
        for j=1:numel(T)
            tic;
            [price(i,j),iv(i,j)] = NumericalIntegrationRoughHeston(s_0,...
                               v_0,alpha,lambda,v_bar,xi,rho,call,K(i),...
                               T(j),'r',r,'q',q,'beta',beta,'N',N,...
                               'ubound',ubound,'nmax',nmax,...
                               'midpoint',midpoint,'eps',eps,'dA',dA);
            tt = toc;
            if disp_iter
                disp([num2str(ii), ' / ', num2str(tot_iter), ...
                   ' prices computed (time: ',num2str(tt),' sec.)']);
            end
            ii = ii + 1;
        end
    end
    return;
end

F = s_0*exp((r-q)*T);
Kadj = K ./ F;
k = log(Kadj);

% Compute optimal beta:
if ~exist('beta','var') || isempty(beta)
    T_star = @(u)(MomentExplosionTimeRoughHeston(alpha,lambda,xi,rho,u,...
                                                 nmax)');
    % Remark: Estimated lower critical moment may be biased high:
    lower_critical = GetLowerCriticalMomentRoughHeston(lambda,rho,xi,...
                                                       T_star,T);
    beta = GetOptimalFourierAlpha(phi,k,lower_critical) + 1;
end

% Compute price of call option assuming S(0) = 1, zero interest rates,
% zero dividends and using the adjusted strike:
price = NumericalIntegrationCall(Kadj,phi,beta,midpoint,ubound,dA,eps);

% Convert to put option if needed:
if ~call
    price = price - 1 + Kadj;
end

if price < 0
    error('NumericalIntegrationRoughHeston: Negative price computed. ');
end

% Scale and adjust back:
price = s_0*exp(-q*T).*price;

% Compute implied volatilities:
% iv = blsimpv(s_0,K,r,T,price,'Yield',q,'class',call);

end



In [ ]:
function [price, iv] = NumericalIntegrationRoughHeston(s_0,v_0,alpha,lambda,...
                                                v_bar,xi,rho,call,K,T,...
                                                varargin)
% Description: Computes prices of European call and put options under the
% rough Heston model as presented in e.g. (Gerhold et al., 2019) or (El
% Euch & Rosenbaum, 2018) and (El Euch & Rosenbaum 2019).
%
% The model for the asset price S(t) can be stated as
%
%   dS(t) = S(t)*sqrt(V(t))*dW_1(t)
%   V(t) = V(0) + int_0^t K(t-s) lambda*(v_bar - V(s))ds 
%               + int_0^t K(t-s) xi * sqrt(V(s))dW_2(s)
%
% with dW_1(t)dW_2(t) = rho dt and K(t) = (1/gamma(alpha))*t^(alpha-1).
% 
% We assume 1/2 < alpha <= 1 and V(0), lambda, v_bar, xi > 0 and 
% -1 <= rho < 0.
%
% To compute prices we use numerical integration via a Fourier transform
% as presented in e.g. (Gerhold et al., 2019) with an optimal integration 
% contour as proposed in (Lord & Kahl, 2006). To compute the characteristic
% function for the integration we solve a Volterra integral equation which 
% appears in e.g. (Gerhold et al., 2019) using the scheme from 
% (Diethelm, 2004).
%
% Parameters:
%   s_0:        [1x1 real] Initial value of asset price.
%   v_0:        [1x1 real] Initial value of variance process.
%   alpha:      [1x1 real] See equations above.
%   lambda:     [1x1 real] See equations above.
%   v_bar:      [1x1 real] See equations above.
%   xi:         [1x1 real] See equations above.
%   rho:        [1x1 real] See equations above.
%   call:       [1x1 boolean] true (call option), false (put option).
%   K:          [Nx1 real] Strike price(s).
%   T:          [Mx1 real] Expiration(s).
%   r:          [1x1 real] Risk-free interest rate.
%   q:          [1x1 real] Dividend yield.
%   beta:       [1x1 real, optional (default = see text)] Parameter for 
%               shifting the integration contour ideally to lower the 
%               oscillations of the integrand. Must be s.t. the beta'th 
%               moment of S(T) is finite (user responsibility). Default is 
%               an optimal value computed along the lines of 
%               (Lord & Kahl, 2006).
%   N:          [1x1 integer, optional(default=250)] Number of steps used 
%               to solve the Volterra integral equation.
%   ubound:     [1x1 real, optional(default=2000)] Upper truncation of 
%               Fourier integral.
%   midpoint:   [1x1 real, optional(default=10)] Used for multi-domain 
%               integration, see function 'NumericalIntegrationCall' and 
%               (Zhu, 2010).
%   dA:         [1x1 real, optional(default=50)] Same as above.
%   eps:        [1x1 real, optional(default=10^(-6))] Same as above.
%   nmax:       [1x1 integer, optional(default=200)] Number of terms in 
%               recursive scheme to compute moment explosion times which 
%               are needed when searching for an optimal contour 
%               shift, i.e. an optimal 'beta'.
%   disp_iter:  [1x1 boolean, optional(false)] If true and multiple prices 
%               are requested we show the time elapsed of each iteration.
% 
% Output: 
%   price: [NxM real] Prices of call or put options.
%   iv:    [NxM real] Black-scholes implied volatilities.
%
% Example: [price, iv] = NumericalIntegrationRoughHeston(100,0.2^2,0.6,2,...
%                                                  0.2^2,0.2,-0.8,true,...
%                                                  [90;100;110],[0.5;1])
%
% References:
%   - Stefan Gerhold, Christoph Gerstenecker, Arpad Pinter, Moment
%   explosions in the rough Heston model, Decisions in Economics and
%   Finance (2019) 42:575-608.
%
%   - Omar El Euch and Mathieu Rosenbaum, Perfect hedging in rough Heston
%   models, Mathematical Finance 28(6), 3813-3856, 2018.
%
%   - Omar El Euch and Mathieu Rosenbaum, The characteristic function of
%   rough Heston models, Mathematical Finance 29(1), 3-38, 2019.
%
%   - Roger lord and Christian Kahl, Optimal Fourier inversion in 
%   semi-analytical option pricing, Tinbergen Institute Discussion Paper 
%   No. 2006-066/2. Available at SRRN:
%   https://papers.ssrn.com/sol3/papers.cfm?abstract_id=921336.
% 
%   - Kai Diethelm, Neville J. Ford and Alan D. Freed, Detailed error
%   analysis for a fractional ADAMs method, Numerical Algorithms 36:31-52,
%   2004.
%

% Parse and check inputs:
p = inputParser;
addOptional(p,'r',0);
addOptional(p,'q',0);
addOptional(p,'beta',[]);
addOptional(p,'N',250);
addOptional(p,'ubound',2000);
addOptional(p,'midpoint',10);
addOptional(p,'dA',50);
addOptional(p,'eps',10^(-6));
addOptional(p,'nmax',200);
addOptional(p,'disp_iter',false);
parse(p,varargin{:});
r = p.Results.r;
q = p.Results.q;
beta = p.Results.beta;
N = p.Results.N;
ubound = p.Results.ubound;
midpoint = p.Results.midpoint;
dA = p.Results.dA;
eps = p.Results.eps;
nmax = p.Results.nmax;
disp_iter = p.Results.disp_iter;

if rho >= 0
    error('NumericalIntegrationRoughHeston: rho >= 0 is current not supported.');
end

if alpha > 1 || alpha <= 1/2
    error('NumericalIntegrationRoughHeston: Alpha must lie in (1/2,1].');
end

if any([v_0,lambda,v_bar,xi]) <= 0
    error(['NumericalIntegrationRoughHeston: v_0, lambda, v_bar and xi ',...
           'must be strictly positive.']);
end


if size(K,2) > 1 || size(T,2) > 1
    error(['NumericalIntegrationRoughHeston: Strike and expiration vectors ', ...
        'must be column vectors.']);
end

% Define characteristic function:
phi = @(u)(MomentGeneratingFunctionRoughHeston(v_0,alpha,lambda,v_bar,...
                                               xi,rho,T,1i*u,N));

% Vectorize by recursive calls:
tot_iter = numel(K)*numel(T);
if tot_iter > 1
    [price,iv] = deal(NaN(size(K,1),size(T,1)));
    ii = 1;
    for i=1:numel(K)
        for j=1:numel(T)
            tic;
            [price(i,j),iv(i,j)] = NumericalIntegrationRoughHeston(s_0,...
                               v_0,alpha,lambda,v_bar,xi,rho,call,K(i),...
                               T(j),'r',r,'q',q,'beta',beta,'N',N,...
                               'ubound',ubound,'nmax',nmax,...
                               'midpoint',midpoint,'eps',eps,'dA',dA);
            tt = toc;
            if disp_iter
                disp([num2str(ii), ' / ', num2str(tot_iter), ...
                   ' prices computed (time: ',num2str(tt),' sec.)']);
            end
            ii = ii + 1;
        end
    end
    return;
end

F = s_0*exp((r-q)*T);
Kadj = K ./ F;
k = log(Kadj);

% Compute optimal beta:
if ~exist('beta','var') || isempty(beta)
    T_star = @(u)(MomentExplosionTimeRoughHeston(alpha,lambda,xi,rho,u,...
                                                 nmax)');
    % Remark: Estimated lower critical moment may be biased high:
    lower_critical = GetLowerCriticalMomentRoughHeston(lambda,rho,xi,...
                                                       T_star,T);
    beta = GetOptimalFourierAlpha(phi,k,lower_critical) + 1;
end

% Compute price of call option assuming S(0) = 1, zero interest rates,
% zero dividends and using the adjusted strike:
price = NumericalIntegrationCall(Kadj,phi,beta,midpoint,ubound,dA,eps);

% Convert to put option if needed:
if ~call
    price = price - 1 + Kadj;
end

if price < 0
    error('NumericalIntegrationRoughHeston: Negative price computed. ');
end

% Scale and adjust back:
price = s_0*exp(-q*T).*price;

% Compute implied volatilities:
iv = blsimpv(s_0,K,r,T,price,'Yield',q,'class',call);

end



In [ ]:
function [price, integrand] = NumericalIntegrationCall(K,phi,beta,midpoint,...
                                                 ubound,dA,eps)
% Description: Computes the price of a European call option using 
% numerical integration. We assume zero interest rates and dividends and  
% that S(0) = 1 where S(t) is the time t price of the underlying asset.
% 
% Parameters:
%   call:     [1x1 boolean] true (call option), false (put option).
%   K:        [1x1 real] Strike price.
%   T:        [1x1 real] Expiration.
%   phi:      [function] Characteristic function of log(S(T)). Must be 
%             vectorized.
%   beta:     [1x1 real] Parameter for shifting the contour of the pricing 
%             integral, same notation as in (Gerhold et al., 2019). It is 
%             the user's responsibility to ensure that the beta'th moment 
%             of the underlying asset exists so the computations are valid.
%   midpoint: [1x1 real] Midpoint of multi-domain integration, see function
%             'MultiDomainIntegration' and (Zhu, 2010).
%   ubound:   [1x1 real] Upper truncation of integral.
%   dA:       [1x1 real] Sub-interval step size for multi-domain 
%             integration, see function 'MultiDomainIntegration' and 
%             (Zhu, 2010). Will be rounded down to nearest value s.t. 
%             midpoint - ubound is divisible by dA.
%   eps:      [1x1 real] Tolerance of multi-domain integration, see 
%             function 'MultiDomainIntegration' and (Zhu, 2010).
%
% Output: 
%   price:      [1x1] Price of call option.
%   integrand:  [function] Function to be integrated. Added for inspection.
%
% References: 
%   - Stefan Gerhold, Christoph Gerstenecker, Arpad Pinter, Moment
%   explosions in the rough Heston model, Decisions in Economics and
%   Finance (2019) 42:575-608.
%
%   - Omar El Euch and Mathieu Rosenbaum, Perfect hedging in rough Heston
%   models, Mathematical Finance 28(6), 3813-3856, 2018.
%
%   - Omar El Euch and Mathieu Rosenbaum, The characteristic function of
%   rough Heston models, Mathematical Finance 29(1), 3-38, 2019.
%
%   - Roger lord and Christian Kahl, Optimal Fourier inversion in 
%   semi-analytical option pricing, Tinbergen Institute Discussion Paper 
%   No. 2006-066/2. Available at SRRN:
%   https://papers.ssrn.com/sol3/papers.cfm?abstract_id=921336.
%
%   - Jianwei Zhu, Applications of Fourier Transform to Smile Modelling,
%   Springer, 2010.
%

    s0 = 1;
    k = log(K);
    alpha = beta - 1;
    psi = @(v) (  phi( v - (alpha + 1)*1i  ) ./ ...
        ( alpha^2 + alpha - v.^2 + 1i*(2*alpha + 1).*v)  );
    integrand = @(v)(real(psi(v).*exp(-1i*v*k')).*(exp(-alpha*k')./pi));
    integrand = @(v)(integrand(v')');
    
    % Remark: We use multi-domain integration to avoid evaluating the 
    % characteristic function at unnecessarily extreme values which  
    % can cause NaNs to appear for some implementations:
    
    N = ceil((ubound - midpoint)/dA);
    price = MultiDomainIntegration(integrand,0,midpoint,ubound,N,eps);
    
    % Correct for residue depending on alpha - see e.g. equation (7) 
    % appearing in (Lord & Kahl, 2006):
    R = 0;
    if alpha <= 0
        R = R + s0;
    end
    if alpha <= -1
        R = R - K;
    end
    if alpha==0
        R = R - 0.5*s0;
    end
    if alpha==-1
        R = R + 0.5*K;
    end
    price = price + R;
    
end



In [ ]:
function [ Gamma_New , F_New ] = num_IPCA_estimate_ALS( Gamma_Old , W , X , Nts , varargin )
% [ Gamma_New , F_New ] = num_IPCA_estimate_ALS( Gamma_Old , W , X , Nts )
% [ Gamma_New , F_New ] = num_IPCA_estimate_ALS( Gamma_Old , W , X , Nts , PSF )
% inputs
% - Gamma_Old : LxK matrix of previous iteration's GammaBeta estimate
% - W : LxLxT array of Z(:,:,t)'*Z(:,:,t)/Nts(t)
% - X : LxT array of Z(:,:,t)'*Y(:,t)/Nts(t)
% - Nts : 1xT vector of cross-sectional size [typically found as sum(LOC)]
% - (optional) PSF : Kadd x T Pre-Specified Factors
%
% outputs
% - Gamma_New : LxK matrix of this iteration's GammaBeta estimate
% - F_New : KxT matrix of this iteration's Factor estimate
%
% Imposes identification assumption on Gamma_New and F_New: Gamma_New is orthonormal
% matrix and F_New has positive mean (taken across the T columns)
%
% When nargin>4, estimates a LxKtilde Gamma_New and a Ktilde x T F_New, where
% Ktilde=K+Kadd
% The first K columns of Gamma_New is GammaBeta.
% The last Kadd columns of Gamma_New is GammaDelta.
% We continue to only return the KxT factor estimate in F_New
%
% When nargin<=4, Ktilde=K and GammaDelta is not estimated

if nargin>4 & ~isempty(varargin{1})
    PSF = varargin{1};
    PSF_version = true;
    [Kadd,Tadd] = size(PSF);
else
    PSF_version = false;
end

T = length(Nts);
if PSF_version % pre-specified factors in the model
    [L,Ktilde] = size(Gamma_Old);
    K = Ktilde-Kadd;
else % no pre-specified factors in the model
    [L,K] = size(Gamma_Old);
    Ktilde = K;
end

F_New = [];
if K>0 % this could run using only prespecified factors
    
if PSF_version % pre-specified factors in the model
    F_New = nan(K,T);
    for t=1:T
        F_New(:,t) = ( Gamma_Old(:,1:K)'*W(:,:,t)*Gamma_Old(:,1:K) )\...
            ( Gamma_Old(:,1:K)'*( X(:,t)-W(:,:,t)*Gamma_Old(:,K+1:Ktilde)*PSF(:,t) ) );
    end
else % no pre-specified factors in the model
    F_New = nan(K,T);
    for t=1:T
        F_New(:,t) = ( Gamma_Old'*W(:,:,t)*Gamma_Old )\( Gamma_Old'*X(:,t) );
    end
end

end


Numer = zeros(L*Ktilde,1);
Denom = zeros(L*Ktilde);
if PSF_version % pre-specified factors in the model
    if K>0
    for t=1:T
        Numer = Numer + kron( X(:,t)        , [F_New(:,t);PSF(:,t)]                            )*Nts(t);
        Denom = Denom + kron( W(:,:,t)      , [F_New(:,t);PSF(:,t)]*[F_New(:,t);PSF(:,t)]'     )*Nts(t);
    end
    else
    for t=1:T
        Numer = Numer + kron( X(:,t)        , [PSF(:,t)]                            )*Nts(t);
        Denom = Denom + kron( W(:,:,t)      , [PSF(:,t)]*[PSF(:,t)]'     )*Nts(t);
    end
    end
else % no pre-specified factors in the model
    for t=1:T
        Numer = Numer + kron( X(:,t)        , F_New(:,t)                )*Nts(t);
        Denom = Denom + kron( W(:,:,t)      , F_New(:,t)*F_New(:,t)'    )*Nts(t);
    end
end
Gamma_New_trans_vec = Denom\Numer;
Gamma_New_trans     = reshape(Gamma_New_trans_vec,Ktilde,L);
Gamma_New           = Gamma_New_trans';

% GammaBeta orthonormal and F_New Orthogonal
if K>0
R1                  = chol(Gamma_New(:,1:K)'*Gamma_New(:,1:K),'upper');
[R2,~,~]            = svd(R1*F_New*F_New'*R1');
Gamma_New(:,1:K)    = (Gamma_New(:,1:K)/R1)*R2;
F_New               = R2\(R1*F_New);
end

% Sign convention on GammaBeta and F_New
if K>0
sg = sign(mean(F_New,2));
sg(sg==0)=1; % if mean zero, do not flip signs of anything
Gamma_New(:,1:K) = Gamma_New(:,1:K).*sg';
F_New = F_New .* sg;
end

end



In [ ]:
classdef NthToDefaultSwap < H_RootObject
    % FixedRateLeg
    % properties: fixedSchedule 
    
    properties(SetAccess = public)
        N
        type
        engineType
        nominal
        fixedSchedule
        spread
        fixedDayCounter
        fixedConvention
        
        settlesAccrual
        paysAtDefaultTime
        
        protectionStartDate
        
        fixedRateLeg
        upfrontPayment
        
        NPV;
        fairRate;
       
    end
    
    methods
        % constructor
        function nthToDefaultSwap = NthToDefaultSwap(N,type,engineType,nominal,fixedSchedule,spread,fixedDayCounter, ...
                                      fixedConvention,settlesAccrual,paysAtDefaultTime,protectionStartDate)
            
            nthToDefaultSwap.N = N;
            nthToDefaultSwap.type = type;
            nthToDefaultSwap.engineType = engineType;
            nthToDefaultSwap.nominal = nominal;
            nthToDefaultSwap.fixedSchedule = fixedSchedule;
            nthToDefaultSwap.spread = spread;
            nthToDefaultSwap.fixedDayCounter = fixedDayCounter;
            nthToDefaultSwap.fixedConvention = fixedConvention;
            
            nthToDefaultSwap.settlesAccrual = settlesAccrual;
            nthToDefaultSwap.paysAtDefaultTime = paysAtDefaultTime;
            
            nthToDefaultSwap.protectionStartDate = protectionStartDate;
            
            % bug fixed 20180913
            % when yield curve building, fixedLeg's payment date convention
            % is given as floatingLeg's payment date convention in quantlib
            % !!!!
            
%             vanillaSwap.fixedLeg = FixedRateLeg(fixedSchedule,nominal,fixedRate,fixedDayCounter,fixedSchedule.convention);
            nthToDefaultSwap.fixedRateLeg = FixedRateLeg(fixedSchedule,nominal,spread,fixedDayCounter,fixedConvention);
            
            nthToDefaultSwap.upfrontPayment = SimpleCashflow(0.0,fixedSchedule.dates{1});
            
            
            
        end
        
        function out = StartDate(nthToDefaultSwap)
            d1 = nthToDefaultSwap.fixedRateLeg.StartDate();
            out = d1;

        end
        
        function out = EndDate(nthToDefaultSwap)
            d1 = nthToDefaultSwap.fixedRateLeg.EndDate();
            out = d1;

        end
        
        
        function    defaultProb = DefaultProbFromDate(nthToDefaultSwap,oneFactorGaussianCopula,survProbCurveList, h_date1,h_date2)
            if nargin > 4
                t1 = DateDiff(h_date1,survProbCurveList{1}.params('asOfDate'))/365.0;
                t2 = DateDiff(h_date2,survProbCurveList{1}.params('asOfDate'))/365.0;
                
                defaultProbList1 = zeros(length(survProbCurveList),1);
                defaultProbList2 = zeros(length(survProbCurveList),1);
                
                for i=1:length(defaultProbList1)
                    defaultProbList1(i) = survProbCurveList{i}.DefaultProb(t1);
                    defaultProbList2(i) = survProbCurveList{i}.DefaultProb(t2);
                end
                
                defaultProb1 = oneFactorGaussianCopula.Integrate(nthToDefaultSwap.N,defaultProbList1);
                defaultProb2 = oneFactorGaussianCopula.Integrate(nthToDefaultSwap.N,defaultProbList2);
                
                defaultProb = defaultProb2 - defaultProb1;
                % gaussian copula part
                
%                 defaultProb = survProbCurve.DefaultProb(t1,t2);
            else
                t1 = DateDiff(h_date1,survProbCurveList{1}.params('asOfDate'))/365.0;
%                 defaultProb = survProbCurve.DefaultProb(t1);
                defaultProbList = zeros(length(survProbCurveList),1);
                for i=1:length(defaultProbList)
                    defaultProbList(i) = survProbCurveList{i}.DefaultProb(t1);
                end
                
                defaultProb = oneFactorGaussianCopula.Integrate(nthToDefaultSwap.N,defaultProbList);
                 % gaussian copula part
                 
            end
        end
        
        function    survProb = SurvProbFromDate(nthToDefaultSwap,oneFactorGaussianCopula,survProbCurveList, h_date)
            
            defaultProb = nthToDefaultSwap.DefaultProbFromDate(oneFactorGaussianCopula,survProbCurveList, h_date);
            survProb = 1.0 - defaultProb;
        end
        
        function out = Calculate(nthToDefaultSwap,recoveryRate,defaultCorrelation,survProbCurveList,discountCurve)
            params = containers.Map;
            params('correlation') = defaultCorrelation;
            oneFactorGaussianCopula = OneFactorGaussianCopula(params);
        
            if strcmp(nthToDefaultSwap.engineType,'Integrate')

                out = Calculate_Integral(nthToDefaultSwap,recoveryRate,oneFactorGaussianCopula,survProbCurveList,discountCurve);
            else
                out = Calculate_Mid(nthToDefaultSwap,recoveryRate,oneFactorGaussianCopula,survProbCurveList,discountCurve);
            end
        end
        
        function out = Calculate_Mid(nthToDefaultSwap,recoveryRate,oneFactorGaussianCopula,survProbCurveList,discountCurve)
            % MidPointCDS Engine !
            
            % consistency check !
            if DateDiff(survProbCurveList{1}.asOfDate,discountCurve.asOfDate) ~= 0
                error('inconsistency ! survProbCurve and discountCurve asOfDate should be the same !')
            end
            
            evaluationDate = discountCurve.asOfDate;
            upfPV01 = 0.0;
            upfrontNPV = 0.0;
            %upfront flow npv has not occured yet  (on-the-run)
            if DateDiff(nthToDefaultSwap.upfrontPayment.paymentDate,evaluationDate) > 0
                effectiveUpFrontDate = nthToDefaultSwap.protectionStartDate;
                upfPV01 = nthToDefaultSwap.SurvProbFromDate(oneFactorGaussianCopula,survProbCurveList,effectiveUpFrontDate) * ...
                          discountCurve.DFFromDate(nthToDefaultSwap.upfrontPayment.paymentDate);
                
                upfrontNPV = upfPV01 * nthToDefaultSwap.upfrontPayment.amount;
            end
            
            couponLegNPV = 0.0;
            defaultLegNPV = 0.0;
            
            legSize = length(nthToDefaultSwap.fixedRateLeg.couponRates);
            for i=1:legSize
                fixedRateCoupon = nthToDefaultSwap.fixedRateLeg.couponRates{i};
                if DateDiff(fixedRateCoupon.paymentDate,evaluationDate) <= 0
                    continue;
                end
                
                paymentDate = fixedRateCoupon.paymentDate;
                startDate = fixedRateCoupon.accrualStartDate;
                endDate = fixedRateCoupon.accrualEndDate;
                
                if i==1
                    startDate = nthToDefaultSwap.protectionStartDate;
                end
                
                if DateDiff(startDate,evaluationDate) <= 0 && DateDiff(evaluationDate,endDate) <= 0
                    effectiveStartDate = evaluationDate;
                else
                    effectiveStartDate = startDate;
                end
                
                defaultDate = effectiveStartDate.AddDate(floor(DateDiff(endDate,effectiveStartDate)/2),'day');
                
                S = nthToDefaultSwap.SurvProbFromDate(oneFactorGaussianCopula,survProbCurveList,paymentDate);
                P = nthToDefaultSwap.DefaultProbFromDate(oneFactorGaussianCopula,survProbCurveList,effectiveStartDate,endDate);
                
                % fixedRate payment in case of survival
                couponLegNPV = couponLegNPV + S * fixedRateCoupon.nominal * fixedRateCoupon.Amount() * ...
                                                  discountCurve.DFFromDate(paymentDate);
                
                % accrual cashflow in case of default
                if nthToDefaultSwap.settlesAccrual
                    if nthToDefaultSwap.paysAtDefaultTime
                        couponLegNPV = couponLegNPV + P * fixedRateCoupon.nominal * fixedRateCoupon.AccruedAmount(defaultDate)* ...
                                       discountCurve.DFFromDate(defaultDate);
                    else
                        couponLegNPV = couponLegNPV + P * fixedRateCoupon.nominal * fixedRateCoupon.Amount()* ...
                                       discountCurve.DFFromDate(paymentDate);
                    end
                end
                
                % at default cashflow
                claim = nthToDefaultSwap.nominal * (1.0 - recoveryRate);
                    
                if nthToDefaultSwap.paysAtDefaultTime
                    defaultLegNPV = defaultLegNPV + P * claim * discountCurve.DFFromDate(defaultDate);
                else
                    defaultLegNPV = defaultLegNPV + P * claim * discountCurve.DFFromDate(paymentDate);
                end
            end

            upfrontSign = 1.0;
            if strcmp(nthToDefaultSwap.type,'ProtectionBuyer')
                couponLegNPV = couponLegNPV * -1.0;
                upfrontNPV = upfrontNPV * -1.0;
            else
                defaultLegNPV = defaultLegNPV * -1.0;
            end
            
            out.NPV = defaultLegNPV + couponLegNPV + upfrontNPV;
            out.fairSpread =  -1.0 * defaultLegNPV * nthToDefaultSwap.spread/couponLegNPV;
            
            
        end
        
        function out = Calculate_Integral(nthToDefaultSwap,recoveryRate,oneFactorGaussianCopula,survProbCurveList,discountCurve)
            % MidPointCDS Engine !
            
            % consistency check !
            if DateDiff(survProbCurveList{1}.asOfDate,discountCurve.asOfDate) ~= 0
                error('inconsistency ! survProbCurve and discountCurve asOfDate should be the same !')
            end
            
            evaluationDate = discountCurve.asOfDate;
            upfPV01 = 0.0;
            upfrontNPV = 0.0;
            %upfront flow npv has not occured yet  (on-the-run)
            if DateDiff(nthToDefaultSwap.upfrontPayment.paymentDate,evaluationDate) > 0
                effectiveUpFrontDate = nthToDefaultSwap.protectionStartDate;
                upfPV01 = nthToDefaultSwap.SurvProbFromDate(oneFactorGaussianCopula,survProbCurveList,effectiveUpFrontDate) * ...
                          discountCurve.DFFromDate(nthToDefaultSwap.upfrontPayment.paymentDate);
                
                upfrontNPV = upfPV01 * nthToDefaultSwap.upfrontPayment.amount;
            end
            
            couponLegNPV = 0.0;
            defaultLegNPV = 0.0;
            
            legSize = length(nthToDefaultSwap.fixedRateLeg.couponRates);
            for i=1:legSize
                fixedRateCoupon = nthToDefaultSwap.fixedRateLeg.couponRates{i};
                if DateDiff(fixedRateCoupon.paymentDate,evaluationDate) <= 0
                    continue;
                end
                
                paymentDate = fixedRateCoupon.paymentDate;
                startDate = fixedRateCoupon.accrualStartDate;
                endDate = fixedRateCoupon.accrualEndDate;
                
                if i==1
                    startDate = nthToDefaultSwap.protectionStartDate;
                end
                
                if DateDiff(startDate,evaluationDate) <= 0 && DateDiff(evaluationDate,endDate) <= 0
                    effectiveStartDate = evaluationDate;
                else
                    effectiveStartDate = startDate;
                end
                
%                 defaultDate = effectiveStartDate.AddDate(floor(DateDiff(endDate,effectiveStartDate)/2),'day');
                
                S = nthToDefaultSwap.SurvProbFromDate(oneFactorGaussianCopula,survProbCurveList,paymentDate);
%                 P = survProbCurve.DefaultProbFromDate(effectiveStartDate,endDate);
                
                % fixedRate payment in case of survival
                couponLegNPV = couponLegNPV + S * fixedRateCoupon.nominal * fixedRateCoupon.Amount() * ...
                                                  discountCurve.DFFromDate(paymentDate);
                
                date0 = effectiveStartDate;
                stepSize = 7;
                date1 = date0.AddDate(stepSize,'day');
                
                %date1 should be earlier than endDate
                % forward generation of the schedule is dangerous!!
                % check for the theta in the future
                
                if DateDiff(date1,endDate) > 0
                    date1 = endDate;
                end
                    
%                 P0 = survProbCurve.DefaultProbFromDate(date0);
                P0 = nthToDefaultSwap.DefaultProbFromDate(oneFactorGaussianCopula,survProbCurveList,date0);
                endDiscount = discountCurve.DFFromDate(paymentDate);
                
                while DateDiff(date0,endDate) < 0
                    if nthToDefaultSwap.paysAtDefaultTime
                        B =  discountCurve.DFFromDate(date1);
                    else
                        B = endDiscount;
                    end
                    
%                     P1 = survProbCurve.DefaultProbFromDate(date1);
                    P1 = nthToDefaultSwap.DefaultProbFromDate(oneFactorGaussianCopula,survProbCurveList,date1);
                    dP = P1 - P0;
                    
                    % accrual cashflow in case of default
                    if nthToDefaultSwap.settlesAccrual
                        if nthToDefaultSwap.paysAtDefaultTime
                            couponLegNPV = couponLegNPV + dP * fixedRateCoupon.nominal * fixedRateCoupon.AccruedAmount(date1)* ...
                                           B;
                        else
                            couponLegNPV = couponLegNPV + dP * fixedRateCoupon.nominal * fixedRateCoupon.Amount()* ...
                                           B;
                        end
                    end
                    
                    % at default cashflow
                    claim = nthToDefaultSwap.nominal * (1.0 - recoveryRate);

                    if nthToDefaultSwap.paysAtDefaultTime
                        defaultLegNPV = defaultLegNPV + dP * claim * B;
                    else
                        defaultLegNPV = defaultLegNPV + dP * claim * B;
                    end
                    
                    % setup for next time around the loop
                    P0 = P1;
                    date0 = date1;
                    date1 = date0.AddDate(stepSize,'day');
                    if DateDiff(date1,endDate) > 0
                        date1 = endDate;
                    end
                    
                end
                
                
                
            end

            upfrontSign = 1.0;
            if strcmp(nthToDefaultSwap.type,'ProtectionBuyer')
                couponLegNPV = couponLegNPV * -1.0;
                upfrontNPV = upfrontNPV * -1.0;
            else
                defaultLegNPV = defaultLegNPV * -1.0;
            end
            
            out.NPV = defaultLegNPV + couponLegNPV + upfrontNPV;
            out.fairSpread =  -1.0 * defaultLegNPV * nthToDefaultSwap.spread/couponLegNPV;
            
            
        end
        
        
    end
    
end



In [ ]:
function [c,mv] = normAm(A,m)
%NORMAM   Estimate of 1-norm of power of matrix.
%   NORMAM(A,m) estimates norm(A^m,1).
%   If A has nonnegative elements the estimate is exact.
%   [C,MV] = NORMAM(A,m) returns the estimate C and the number MV of
%   matrix-vector products computed involving A or A^*.

%   Reference: A. H. Al-Mohy and N. J. Higham, A New Scaling and Squaring
%   Algorithm for the Matrix Exponential, SIAM J. Matrix Anal. Appl. 31(3):
%   970-989, 2009.

%   Awad H. Al-Mohy and Nicholas J. Higham, September 7, 2010.

t = 1; % Number of columns used by NORMEST1.

n = length(A);
if isequal(A,abs(A))
    e = ones(n,1);
    for j=1:m         % for positive matrices only
        e = A'*e;
    end
    c = norm(e,inf);
    mv = m;
else
    [c,v,w,it] = normest1(@afun_power,t);
    mv = it(2)*t*m;
end

  function Z = afun_power(flag,X)
       %AFUN_POWER  Function to evaluate matrix products needed by NORMEST1.

       if isequal(flag,'dim')
          Z = n;
       elseif isequal(flag,'real')
          Z = isreal(A);
       else

          [p,q] = size(X);
          if p ~= n, error('Dimension mismatch'), end

          if isequal(flag,'notransp')
             for i = 1:m, X = A*X; end
          elseif isequal(flag,'transp')
             for i = 1:m, X = A'*X; end
          end

          Z = X;

       end

  end
end


In [ ]:
function delta = NormalVanillaFwdDelta(txDay,fwd,fwdStrike,sig,CPFlag)
        %NormalVanilla:a function that computes a blackCallAndPut price
        %arguments
        %txDay =(days)time to reset
        %tpDay =(days)time to pay
        %sig= implied volatility
        %strike  = caplet strike
        %CPFlag = Call Or Put
%             fwd = eqBlack.Fwd(spot,txDay);
            tx = txDay/365.0;
%             tp = tpDay/365.0;
%             df = eqBlack.zeroCurve.DF(tp);
            d1= (fwd - fwdStrike)/(sig*sqrt(tx));
%             d2 = d1 -sig*sqrt(tx);
            if strcmp(CPFlag,'C')
%                 vanilla = df*(fwd*H_ncdf(d1)-strike*H_ncdf(d2));
%                 vanilla = blackPrice(fwd,fwdStrike,sig,tx,1);
                delta = H_ncdf(d1);
            elseif strcmp(CPFlag,'P')
%                 vanilla = df*(-fwd*H_ncdf(-d1)+strike*H_ncdf(-d2));
%                 vanilla = blackPrice(fwd,fwdStrike,sig,tx,-1);
                delta = -1.0*H_ncdf(-d1);
            end
            
        end


In [ ]:
function vanilla = NormalVanillaFwd(txDay,fwd,fwdStrike,sig,CPFlag)
    %NormalVanilla:a function that computes a blackCallAndPut price
    %arguments
    %txDay =(days)time to reset
    %tpDay =(days)time to pay
    %sig= implied volatility
    %strike  = caplet strike
    %CPFlag = Call Or Put
    tx = txDay/365.0;
    d1= (fwd - fwdStrike)/(sig*sqrt(tx));
    if strcmp(CPFlag,'C')
        vanilla = (fwd-fwdStrike)*H_ncdf(d1) + sig*sqrt(tx)*H_ndf(d1);
    elseif strcmp(CPFlag,'P')
        vanilla = -1.0*(fwd-fwdStrike)*H_ncdf(-d1) + sig*sqrt(tx)*H_ndf(d1);
    end

end



In [ ]:
%NEARCORR_TEST  Compare g02aa and nearcorr.
 
rng(10)                                  % Seed random number generators.
n = 100;
n = 1399;

A = gallery('randcorr',n);               % Random correlation matrix. 
E  = randn(n)*1e-1;  A = A + (E + E')/2; % Perturb it.
tol = 1e-10;
 
% A = cor1399; tol = 1e-4;
 
% fprintf('g02aa:\n')
maxits = int64(-1);  % For linear equation solver.
maxit = int64(-1);   % For Newton iteration.
% tic
% [~,X1,iter1,feval,nrmgrd,ifail] = g02aa(A,'errtol',tol,'maxits',maxits, ...
%                                           'maxit',maxit);
% toc
 
% fprintf('  Newton steps taken: %d\n', iter1);
% fprintf('  Norm of gradient of last Newton step: %6.4f\n', nrmgrd);
% if ifail > 0, fprintf('  g02aa failed with ifail = %g\n', ifail), end
 
fprintf('nearcorr:\n')
tic
[X2,iter2] = nearcorr(A,tol,[],[],[],[],1);
toc
fprintf('  Number of iterations: %d\n', iter2);
 
% fprintf('  Normwise relative difference between computed solutions:')
% fprintf('%9.2e\n', norm(X1-X2,1)/norm(X1,1))

In [ ]:
function [Y,iter] = nearcorr_new_noDykstra(A,pattern,delta,tol,itmax)
% nearcorr_new  Nearest correlation matrix by alternating projections.
%   [Y,ITER] = nearcorr_new(A,PATTERN,DELTA,TOL,ITMAX) finds the
%   nearest correlation matrix Y to a symmetric matrix A by the 
%   alternating projections method.  ITER is the number of iterations taken.
%   Prescribed elements can be kept fixed by specifying them in the
%   matrix PATTERN: the (i,j) element is 1 if the corresponding element 
%   of A is to remain fixed, else zero.
%   By default PATTERN is empty, meaning no elements are fixed.
%   If PATTERN is non-empty then the unit diagonal must be explicitly
%   forced if required.
%   The solution can be forced to be positive definite with smallest
%   eigenvalue at least DELTA, 0 < DELTA <= 1. By default, DELTA = 0.
%   TOL is the convergence tolerance. Default: length(A)*eps.
%   ITMAX is the maximum number of iterations allowed. Default: 100.

%   This is a modified version of nearcorr.m by Nick Higham, from
%   https://nickhigham.wordpress.com/2013/02/13/the-nearest-correlation-matrix/
%   This version removes the support for weights and "highly
%   non-positive definite A" but adds the PATTERN and DELTA options.

%   Nick Higham and Natasa Strabic, 2015.

if ~isequal(A,A'), error('The input matrix must by symmetric.'), end
if nargin < 2, pattern = []; end
if nargin < 3 || isempty(delta), delta = 0; end
if nargin < 4 || isempty(tol), tol = length(A)*eps; end
if nargin < 5, itmax = 100; end

Y = A;
iter = 0;
rel_diffXY = inf;
dS = zeros(size(A));

while rel_diffXY > tol

%     [X,Y,dS] = ap_step(A,Y,dS,pattern,delta);
    [X,Y,dS]  = ap_step_noDykstra(A,Y,dS,pattern,delta);
    rel_diffXY = norm(Y-X,'fro')/norm(Y,'fro');
   
    iter = iter + 1;
    if iter > itmax
%        error(['Stopped after ' num2str(itmax) ' its. Try increasing ITMAX.'])
       disp(['Stopped after ' num2str(itmax) ' its. Try increasing ITMAX.'])
       break;
       
    end
    
end
end

% --------------------------------------------
% Subfunctions

function [X,Y,dS] = ap_step(A,Y,dS,pattern,delta)
% Fixed point iteration for the alternating projections step 
% for the nearest correlation matrix.

R = Y - dS;
X = proj_spd(R,delta);
dS = X - R;
Y = proj_pattern(A,X,pattern);
end

function [X,Y,dS] = ap_step_noDykstra(A,Y,dS,pattern,delta)
% Fixed point iteration for the alternating projections step 
% for the nearest correlation matrix.

% R = Y - dS;
R = Y;

X = proj_spd(R,delta);
dS = X - R;
Y = proj_pattern(A,X,pattern);
end

function X = proj_pattern(A,X,pattern)
% Nearest matrix to X with fixed elements from A specified in
% the positions PATTERN.
 
n = length(A);
if isempty(pattern)
    % Standard NCM, with no fixed elements.  Set unit diagonal.
    X(1:n+1:n^2) = 1;
else
    F = find(pattern); % Locations of fixed elements in A.
    X(F) = A(F);
end
end

function X = proj_spd(A,delta)
% Return the nearest positive semidefinite matrix to A with the smallest
% eigenvalue at least delta.
 
[V,D] = eig(A);
X = V*diag(max(diag(D),delta))*V';
X = (X+X')/2; % Ensure symmetry.
end


In [ ]:
function [Y,iter] = nearcorr_new(A,pattern,delta,tol,itmax)
% nearcorr_new  Nearest correlation matrix by alternating projections.
%   [Y,ITER] = nearcorr_new(A,PATTERN,DELTA,TOL,ITMAX) finds the
%   nearest correlation matrix Y to a symmetric matrix A by the 
%   alternating projections method.  ITER is the number of iterations taken.
%   Prescribed elements can be kept fixed by specifying them in the
%   matrix PATTERN: the (i,j) element is 1 if the corresponding element 
%   of A is to remain fixed, else zero.
%   By default PATTERN is empty, meaning no elements are fixed.
%   If PATTERN is non-empty then the unit diagonal must be explicitly
%   forced if required.
%   The solution can be forced to be positive definite with smallest
%   eigenvalue at least DELTA, 0 < DELTA <= 1. By default, DELTA = 0.
%   TOL is the convergence tolerance. Default: length(A)*eps.
%   ITMAX is the maximum number of iterations allowed. Default: 100.

%   This is a modified version of nearcorr.m by Nick Higham, from
%   https://nickhigham.wordpress.com/2013/02/13/the-nearest-correlation-matrix/
%   This version removes the support for weights and "highly
%   non-positive definite A" but adds the PATTERN and DELTA options.

%   Nick Higham and Natasa Strabic, 2015.

if ~isequal(A,A'), error('The input matrix must by symmetric.'), end
if nargin < 2, pattern = []; end
if nargin < 3 || isempty(delta), delta = 0; end
if nargin < 4 || isempty(tol), tol = length(A)*eps; end
if nargin < 5, itmax = 100; end

Y = A;
iter = 0;
rel_diffXY = inf;
dS = zeros(size(A));

while rel_diffXY > tol

    [X,Y,dS] = ap_step(A,Y,dS,pattern,delta);
%     [X,Y,dS]  = ap_step_noDykstra(A,Y,dS,pattern,delta);
    rel_diffXY = norm(Y-X,'fro')/norm(Y,'fro');
   
    iter = iter + 1;
    if iter > itmax
%        error(['Stopped after ' num2str(itmax) ' its. Try increasing ITMAX.'])
       disp(['Stopped after ' num2str(itmax) ' its. Try increasing ITMAX.'])
       break;
       
    end
    
end
end

% --------------------------------------------
% Subfunctions

function [X,Y,dS] = ap_step(A,Y,dS,pattern,delta)
% Fixed point iteration for the alternating projections step 
% for the nearest correlation matrix.

R = Y - dS;
X = proj_spd(R,delta);
dS = X - R;
Y = proj_pattern(A,X,pattern);
end

function [X,Y,dS] = ap_step_noDykstra(A,Y,dS,pattern,delta)
% Fixed point iteration for the alternating projections step 
% for the nearest correlation matrix.

% R = Y - dS;
R = Y;

X = proj_spd(R,delta);
dS = X - R;
Y = proj_pattern(A,X,pattern);
end

function X = proj_pattern(A,X,pattern)
% Nearest matrix to X with fixed elements from A specified in
% the positions PATTERN.
 
n = length(A);
if isempty(pattern)
    % Standard NCM, with no fixed elements.  Set unit diagonal.
    X(1:n+1:n^2) = 1;
else
    F = find(pattern); % Locations of fixed elements in A.
    X(F) = A(F);
end
end

function X = proj_spd(A,delta)
% Return the nearest positive semidefinite matrix to A with the smallest
% eigenvalue at least delta.
 
[V,D] = eig(A);
X = V*diag(max(diag(D),delta))*V';
X = (X+X')/2; % Ensure symmetry.
end


In [ ]:
function [X,iter] = nearcorr_aa(A,pattern,mMax,itmax,ls_solve,delta,...
                                tol,droptol,AAstart)
%nearcorr_aa     Nearest correlation matrix with Anderson Acceleration.
%   [X,ITER] = nearcorr_aa(A,PATTERN,MMAX,ITMAX,LS_SOLVE,DELTA,...
%                          TOL,DROPTOL,BETA,AASTART)
%   finds the nearest correlation matrix X to a symmetric matrix A by
%   the alternating projections method with Anderson acceleration.
%   ITER is the number of iterations taken.
%   Prescribed elements can be kept fixed by specifying them in the
%   matrix PATTERN: the (i,j) element is 1 if the corresponding element 
%   of A is to remain fixed, else zero. 
%   By default PATTERN is empty, meaning no elements are fixed.
%   If PATTERN is non-empty then the unit diagonal must be explicitly
%   forced if required.
%   The solution can be forced to be positive definite with smallest
%   eigenvalue at least DELTA, 0 < DELTA <= 1. Default: DELTA = 0.
%   MMAX = history length parameter (non-negative integer).
%          Default: 2.  MMAX = 0 means no acceleration.
%   ITMAX = maximum allowable number of iterations.  Default: 100.
%   LS_SOLVE = the least-squares solve method:
%              'u': QR factorization with updating (default),
%              'n': normal equations,
%              'b': MATLAB's backslash.
%   TOL = convergence tolerance. Default: length(A)*eps.
%   DROPTOL = tolerance for dropping stored residual vectors to improve
%             conditioning. If DROPTOL > 0, drop residuals if the
%             condition number exceeds DROPTOL; if DROPTOL <= 0,
%             do not drop residuals. Default: 0.
%   AASTART = acceleration delay factor: If AASTART > 0, start 
%             acceleration when iter = AAstart. Default: 1.

%   Based on the code AndAcc.m by Homer Walker dated 10/14/2011,
%   which is listed in 
%   Homer F. Walker, Anderson acceleration: Algorithms and implementations.
%   Technical Report MS-6-15-50, Mathematical Sciences Department, 
%   Worcester Polytechnic Institute, Worcester, MA 01609, USA, June 2011.
%   This code is specialized for the alternating projections method for the
%   nearest correlation matrix, adds two extra options for solving
%   the least squares problem, and contains many other changes.

%   Nick Higham and Natasa Strabic, 2015.

if ~isequal(A,A'), error('The input matrix must by symmetric.'), end
n = length(A);

if nargin < 2, pattern = []; end
if nargin < 3 || isempty(mMax), mMax = 2; end
if nargin < 4 || isempty(itmax), itmax = 100; end
if nargin < 5 || isempty(ls_solve), ls_solve = 'u'; end
if nargin < 6 || isempty(delta), delta = 0; end
if nargin < 7 || isempty(tol), tol = n*eps; end
if nargin < 8 || isempty(droptol), droptol = 0; end
if nargin < 9, AAstart = 1; end

% Initialize the storage arrays.
DG = []; % Storage of g-value differences.
DF = []; % Storage of f-value differences, needed for 'n' and 'b' options.

% Initialize the number of stored residuals.
mAA = 0;

% Initialization for the nearest corr. matrix problem.
Yin = A;
Sin = zeros(n);

% Top of the iteration loop.
for iter = 1:itmax
    % Apply g
    [Xout,Yout,Sout] = ap_step(A,Yin,Sin,pattern,delta);
    
    % We have to vectorize the output.
    x = [Yin(:); Sin(:)];
    gval = [Yout(:); Sout(:)];

    fval = gval - x;
    
    % Convergence test.
    rel_diffXY = norm(Yout - Xout,'fro')/norm(Yout,'fro');
    if rel_diffXY < tol, break; end
    
    if mMax == 0 || iter < AAstart
    % Without acceleration, update x <- g(x) to obtain the next
    % approximate solution.
        x = gval;
    else
    % Apply Anderson acceleration.
    
    % Update the df vector and the DG array.
    % If the least-squares solves are by normal equations or backslash, we
    % need DF, the matrix of function value differences built in the same
    % way as DG.
        if iter > AAstart
            df = fval-f_old;
            if mAA < mMax
                DG = [DG gval-g_old];
                if ls_solve ~= 'u'
                    DF = [DF df];
                end
            else
                DG(:,1)=[];
                DG(:,mAA) = gval-g_old;
                if ls_solve ~= 'u'
                    DF(:,1) = [];
                    DF(:,mAA) = df;
                end
            end
            mAA = mAA + 1;
        end
        f_old = fval;
        g_old = gval;
        
        if mAA == 0
            % Update x <- g(x) to obtain the next approximate 
            % solution.  This is the x1 = g(x0) line in pseudocode.
            x = gval;
        else
            % Solve least-squares problem and update solution.        
            switch ls_solve
              case 'u' % Use the QR factors.
                if mAA == 1
                % Form the initial QR decomposition.
                    R(1,1) = norm(df);
                    Q = R(1,1)\df;
                else
                % Update the QR decomposition.
                    if mAA > mMax
                    % If the column dimension of Q is mMax, delete
                    % the first column and update the decomposition.
                        [Q,R] = qrdelete(Q,R,1);
                        mAA = mAA - 1;
                    % The following treats the qrdelete quirk described below.
                        if size(R,1) ~= size(R,2),
                            Q = Q(:,1:mAA-1); R = R(1:mAA-1,:);
                        end
                    % Explanation: If Q is not square, then
                    % qrdelete(Q,R,1) reduces the column
                    % dimension of Q by 1 and the column and row
                    % dimensions of R by 1. But if Q *is* square, then
                    % the column dimension of Q is not reduced and only
                    % the column dimension of R is reduced by one. This
                    % is to allow for MATLAB?셲 default "thick" QR
                    % decomposition, which always produces a square Q.
                    end
                    % Now update the QR decomposition to incorporate
                    % the new column.
                    for j = 1:mAA - 1
                        R(j,mAA) = Q(:,j)'*df;
                        df = df - R(j,mAA)*Q(:,j);
                    end
                    R(mAA,mAA) = norm(df);
                    Q = [Q,R(mAA,mAA)\df];
                end

                if droptol > 0
                % Drop residuals to improve conditioning if necessary.
                    condDF = cond(R);
                    while condDF > droptol && mAA > 1
                        fprintf('cond(D) = %e, reducing mAA to %d \n', ...
                                condDF, mAA-1);
                        [Q,R] = qrdelete(Q,R,1);
                        DG = DG(:,2:mAA);
                        mAA = mAA - 1;
                        % Treat the qrdelete quirk described above.
                        if size(R,1) ~= size(R,2),
                            Q = Q(:,1:mAA); R = R(1:mAA,:);
                        end
                        condDF = cond(R);
                    end
                end % if droptolerance

                % Solve the least-squares problem.
                gamma = R\(Q'*fval);

              case 'n' % use normal equations
                  if mAA > mMax, mAA = mAA - 1; end
                  c = DF'*fval;
                  [~,R] = qr(DF,0); % economy size QR
                  y = R'\c;
                  gamma = R\y;

              case 'b' % use backslash
                  if mAA > mMax, mAA = mAA - 1; end
                  gamma = DF\fval;
            end % switch                   
                                      
            % Update the approximate solution.
            x = gval - DG*gamma;            
  
        end % end of least-squares solve, we have the next iterate x
        
    end % end of the acceleration part
    
    % Update the parameters: undo reshape of x to get new Yin, Sin.
    Yin(:) = x(1:n^2);
    Sin(:) = x(n*n+1:2*n^2);

end
% Bottom of the iteration loop.

if rel_diffXY > tol && iter == itmax,
    error(['Stopped after ' num2str(itmax) ' its. Try increasing ITMAX.'])
end

X = Yin;

end

% --------------------------------------------
% Subfunctions
function [Xout,Yout,Sout] = ap_step(A,Yin,Sin,pattern,delta)
% Fixed point iteration for the alternating projections step
% for the nearest correlation matrix.

R = Yin - Sin;
Xout = proj_spd(R,delta);
Sout = Xout - R;
Yout = proj_pattern(A,Xout,pattern);
end

function X = proj_pattern(A,X,pattern)
% Return the nearest matrix to X with fixed elements from A specified in
% the positions PATTERN
 
n = length(A);
if isempty(pattern)
    % Standard NCM, with no fixed elements.  Set unit diagonal.
    X(1:n+1:n^2) = 1;  
else
    F = find(pattern); % Locations of fixed elements in A.
    X(F) = A(F);
end
end

function X = proj_spd(A,delta)
% Return the nearest positive semidefinite matrix to A with smallest
% eigenvalue at least delta.
 
[V,D] = eig(A);
X = V*diag(max(diag(D),delta))*V';
X = (X+X')/2; % Ensure symmetry.
end

In [ ]:
function [X,iter] = nearcorr(A,tol,flag,maxits,n_pos_eig,w,prnt)
%NEARCORR    Nearest correlation matrix.
%   X = NEARCORR(A,TOL,FLAG,MAXITS,N_POS_EIG,W,PRNT)
%   finds the nearest correlation matrix to the symmetric matrix A.
%   TOL is a convergence tolerance, which defaults to 16*EPS.
%   If using FLAG == 1, TOL must be a 2-vector, with first component
%   the convergence tolerance and second component a tolerance
%   for defining "sufficiently positive" eigenvalues.
%   FLAG = 0: solve using full eigendecomposition (EIG).
%   FLAG = 1: treat as "highly non-positive definite A" and solve
%             using partial eigendecomposition (EIGS).
%   MAXITS is the maximum number of iterations (default 100, but may
%   need to be increased).
%   N_POS_EIG (optional) is the known number of positive eigenvalues of A.
%   W is a vector defining a diagonal weight matrix diag(W).
%   PRNT = 1 for display of intermediate output.
 
%   By N. J. Higham, 13/6/01, updated 30/1/13, 15/11/14, 07/06/15.
%   Reference:  N. J. Higham, Computing the nearest correlation
%   matrix---A problem from finance. IMA J. Numer. Anal.,
%   22(3):329-343, 2002.
 
if ~isequal(A,A'), error('A must by symmetric.'), end
if nargin < 2 || isempty(tol), tol = length(A)*eps*[1 1]; end
if nargin < 3 || isempty(flag), flag = 0; end
if nargin < 4 || isempty(maxits), maxits = 100; end
if nargin < 6 || isempty(w), w = ones(length(A),1); end
if nargin < 7, prnt = 1; end
 
n = length(A);
if flag >= 1
   if nargin < 5 || isempty(n_pos_eig)
      [V,D] = eig(A); d = diag(D);
      n_pos_eig = sum(d >= tol(2)*d(n));
   end
   if prnt, fprintf('n = %g, n_pos_eig = %g\n', n, n_pos_eig), end
end
 
X = A; Y = A;
iter = 0;
rel_diffX = inf; rel_diffY = inf; rel_diffXY = inf;
dS = zeros(size(A));
 
w = w(:); Whalf = sqrt(w*w');
 
while max([rel_diffX rel_diffY rel_diffXY]) > tol(1)
 
   Xold = X;
   R = Y - dS;
   R_wtd = Whalf.*R;
   if flag == 0
      X = proj_spd(R_wtd);
   elseif flag == 1
      [X,np] = proj_spd_eigs(R_wtd,n_pos_eig,tol(2));
   end
   X = X ./ Whalf;
   dS = X - R;
   Yold = Y;
   Y = proj_unitdiag(X);
   rel_diffX = norm(X-Xold,'fro')/norm(X,'fro');
   rel_diffY = norm(Y-Yold,'fro')/norm(Y,'fro');
   rel_diffXY = norm(Y-X,'fro')/norm(Y,'fro');
   iter = iter + 1;
   if prnt
      fprintf('%2.0f:  %9.2e  %9.2e  %9.2e', ...
               iter, rel_diffX, rel_diffY, rel_diffXY)
      if flag >= 1, fprintf('  np = %g\n',np), else fprintf('\n'), end
   end
   if iter > maxits
       error(['Stopped after ' num2str(maxits) ' its. Try increasing MAXITS.'])
   end
 
end
 
%%%%%%%%%%%%%%%%%%%%%%%%
function A = proj_spd(A)
%PROJ_SPD
 
if ~isequal(A,A'), error('Not symmetric!'), end
[V,D] = eig(A);
A = V*diag(max(diag(D),0))*V';
A = (A+A')/2; % Ensure symmetry.
 
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
function [A,n_pos_eig_found] = proj_spd_eigs(A,n_pos_eig,tol)
%PROJ_SPD_EIGS
 
if ~isequal(A,A'), error('Not symmetric!'), end
k = n_pos_eig + 10; % 10 is safety factor.
if k > length(A), k = n_pos_eig; end
opts.disp = 0;
[V,D] = eigs(A,k,'LA',opts); d = diag(D);
j = (d > tol*max(d));
n_pos_eig_found = sum(j);
A = V(:,j)*D(j,j)*V(:,j)';  % Build using only the selected eigenpairs.
A = (A+A')/2; % Ensure symmetry.
 
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
function A = proj_unitdiag(A)
%PROJ_SPD
 
n = length(A);
A(1:n+1:n^2) = 1;

In [ ]:
import pandas as pd
import requests
from lxml import html
from tqdm import tqdm

In [ ]:
# 삼성전자
sample_code = '005930'

In [ ]:
# parsing URL
# 우리 컴퓨터 -> [접속] -> 에프앤가이드(Data Source) -> [크롤링/웹 스크래핑] -> 우리 컴퓨터
# client -> request -> [Server] -> response -> client

SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'


In [ ]:
# object -> 데이터 덩어리(추상화)

snap_url = SNAP_URL.format(sample_code)
snap_content = requests.get(snap_url).content # 문자열 binary
snap_tree = html.fromstring(snap_content) # 객체(object)
per = snap_tree.xpath('//*[@id="corp_group2"]/dl[1]/dd')[0].text
per = float(per)

In [ ]:
per

8.15

In [ ]:
ratio_url = RATIO_URL.format(sample_code)
ratio_content = requests.get(ratio_url).content
ratio_tree = html.fromstring(ratio_content)
debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
debt_ratio = float(debt_ratio)

In [ ]:
# stockMkt => KOSPI
# kosdaqMkt => KOSDAQ
# konexMkt => KONEX

In [ ]:
def get_stock_list(market):
    market_code = ''
    if market == 'kospi':
        market_code = 'stockMkt'
    elif market == 'kosdaq':
        market_code = 'kosdaqMkt'
    elif market == 'konex':
        market_code = 'konexMkt'
    kind_url = 'https://kind.krx.co.kr/corpgeneral/corpList.do?method=download&pageIndex=1&currentPageSize=3000&comAbbrv=&beginIndex=&orderMode=3&orderStat=D&isurCd=&repIsuSrtCd=&searchCodeType=&marketType={}&searchType=13&industry=&fiscalYearEnd=all&comAbbrvTmp=&location=all'.format(market_code)                                                                 

    return pd.read_html(kind_url, converters={'종목코드':lambda x: str(x)})[0]


In [ ]:
def converter(x):
    return str(x)

# 람다 함수 == 무명(anonymous) 함수 == 일회용 함수
lambda x: str(x)

<function __main__.<lambda>(x)>

In [ ]:
kospi_df = get_stock_list('kospi')
print(kospi_df.shape)

(829, 9)


In [ ]:
kosdaq_df = get_stock_list('kosdaq')
print(kosdaq_df.shape)

(1632, 9)


In [ ]:
# merge -> SQL Join
# append | kospi_df.append([kosdaq_df])
# concatenate(합치다)
stock_list_df = pd.concat([kospi_df, kosdaq_df] )

In [ ]:
print(stock_list_df.shape)

(2461, 9)


In [ ]:
# stock_list_df['종목코드'].dropna()
stock_list_df = stock_list_df[stock_list_df['종목코드'].notnull()]

In [ ]:
stock_list_df = stock_list_df[~stock_list_df['회사명'].str.contains('스팩|리츠')]
print(stock_list_df.shape)

(2364, 9)


In [ ]:
# list comprehension
stock_list_df.index = [x for x in range(len(stock_list_df))]

In [ ]:
stock_list_df.to_csv('kospi_kosdaq_stock_list.csv', encoding='utf-8', index=True)

In [ ]:
code_list = stock_list_df['종목코드']
code_list

0       100090
1       453340
2       452260
3       450140
4       377740
         ...  
2359    013030
2360    019550
2361    019570
2362    019590
2363    006920
Name: 종목코드, Length: 2364, dtype: object

In [ ]:
sample_df = pd.DataFrame(
    {'005930':['삼성전자', 1, 2], '035720':['카카오', 1, 2], '015720':['카카오', 1, 2], '025720':['카카오', 1, 2]}
).transpose()



In [ ]:
sample_df.columns = ['name', 'PER', 'Debt_ratio']

In [ ]:
sample_df

,name,PER,Debt_ratio
005930,삼성전자,1,2
035720,카카오,1,2
015720,카카오,1,2
025720,카카오,1,2


In [ ]:
def FinanceInfoCrawler(li, df):
    result_dict = {}
    error_codes = []
    
    for code in tqdm(li):
        try:
            # Parsing URL setting
            SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
            RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'

            # company name
            company_name = df[df['종목코드'] == code]['회사명'].values[0]

            # Get PER
            snap_url = SNAP_URL.format(code)
            snap_content = requests.get(snap_url).content
            snap_tree = html.fromstring(snap_content)
            per = snap_tree.xpath('//*[@id="corp_group2"]/dl[1]/dd')[0].text
            per = float(per)

            # Get Debt ratio
            ratio_url = RATIO_URL.format(code)
            ratio_content = requests.get(ratio_url).content
            ratio_tree = html.fromstring(ratio_content)
            debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
            debt_ratio = float(debt_ratio)
            
            result_dict[code] = [company_name, per, debt_ratio]
            
        except (TypeError, IndexError, AttributeError, ValueError):
            pass
#             print(code)
            error_codes.append(code)
    
    # convert dict to DataFrame
    result_df = pd.DataFrame(result_dict)
    
    # transpose DataFrame
    result_df = result_df.transpose()
    
    # Setting column names
    result_df.columns = ['Name', 'PER', 'Debt_ratio']
    
    return result_df, error_codes

In [ ]:
crawling_result_df = FinanceInfoCrawler(code_list[:50],stock_list_df)

100%|██████████| 50/50 [00:46<00:00,  1.07it/s]


In [ ]:
print(crawling_result_df[0].shape)
crawling_result_df[0].head()

(39, 3)


,Name,PER,Debt_ratio
100090,SK오션플랜트,38.11,132.0
377740,바이오노트,1.89,9.8
446070,유니드비티플러스,104.18,11.2
108320,LX세미콘,7.5,35.7
126720,수산인더스트리,6.3,24.7


In [ ]:
# original data
# crawling_result_df

# copy data
copy_df = crawling_result_df[0].copy()

In [ ]:
# PER 10 이하
# 부채비율 50 이하
# (상위) 20개 종목

final_result_df = copy_df[
    (copy_df['PER'] <= 10)&(copy_df['Debt_ratio'] <= 50)&(copy_df['PER'] > 0)
].sort_values(
    by='PER', ascending=True
).iloc[:20]


In [ ]:
import datetime

# 시간까지 포함한 날짜
now = datetime.datetime.now()

final_result_df.to_csv('LowPER_LowDR_{}.csv'.format(now.strftime('%Y%m%d')))

In [ ]:
pd.read_csv('LowPER_LowDR_{}.csv'.format(now.strftime('%Y%m%d')))

,Unnamed: 0,Name,PER,Debt_ratio
0,377740,바이오노트,1.89,9.8
1,137310,에스디바이오센서,2.39,10.7
2,383800,LX홀딩스,3.90,1.5
3,353200,대덕전자,6.16,39.4
4,363280,티와이홀딩스,6.17,47.6
5,126720,수산인더스트리,6.30,24.7
6,108320,LX세미콘,7.50,35.7
